# effects

> Shadow, opacity and other visual effect utilities for Tailwind CSS

In [ ]:
#| default_exp utilities.effects

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, Dict, Any, Union
from cjm_fasthtml_tailwind.core.base import (
    BaseUtility, StandardUtility, BaseFactory, SingleValueFactory,
    combine_classes, TailwindValue, is_custom_property, is_arbitrary_value
)
from cjm_fasthtml_tailwind.builders.colors import ColoredFactory, ColorValue, ColoredUtility
from cjm_fasthtml_tailwind.builders.scales import SimpleFactory

## Box Shadow

Utilities for controlling the box shadow of an element. Box shadows in Tailwind CSS v4 support both shadow size and shadow color customization.

### Shadow Size

Define the available shadow sizes from smallest to largest:

In [ ]:
#| export
# Shadow size values mapped to their CSS variable names
SHADOW_SIZES = {
    "2xs": "shadow-2xs",   # var(--shadow-2xs) - 0 1px rgb(0 0 0 / 0.05)
    "xs": "shadow-xs",     # var(--shadow-xs) - 0 1px 2px 0 rgb(0 0 0 / 0.05)
    "sm": "shadow-sm",     # var(--shadow-sm) - 0 1px 3px 0 rgb(0 0 0 / 0.1), 0 1px 2px -1px rgb(0 0 0 / 0.1)
    "md": "shadow-md",     # var(--shadow-md) - 0 4px 6px -1px rgb(0 0 0 / 0.1), 0 2px 4px -2px rgb(0 0 0 / 0.1)
    "lg": "shadow-lg",     # var(--shadow-lg) - 0 10px 15px -3px rgb(0 0 0 / 0.1), 0 4px 6px -4px rgb(0 0 0 / 0.1)
    "xl": "shadow-xl",     # var(--shadow-xl) - 0 20px 25px -5px rgb(0 0 0 / 0.1), 0 8px 10px -6px rgb(0 0 0 / 0.1)
    "2xl": "shadow-2xl",   # var(--shadow-2xl) - 0 25px 50px -12px rgb(0 0 0 / 0.25)
    "none": "shadow-none"  # 0 0 #0000
}

In [ ]:
#| export
class ShadowUtility(BaseUtility):
    """Utility class for box shadows with size support."""
    
    def __init__(
        self,
        size: Optional[str] = None,  # Shadow size or custom value
    ):
        """Initialize shadow utility with optional size."""
        super().__init__("shadow")
        if size is not None:
            self._value = self._format_value(size)
    
    def _format_value(
        self,
        value: TailwindValue  # The value to format (size or arbitrary)
    ) -> str:  # The formatted value string
        """Format shadow value according to Tailwind conventions."""
        value_str = str(value)
        
        # Check if it's a standard size
        if value_str in SHADOW_SIZES:
            # Return just the size part (e.g., "sm", "lg")
            return value_str
        
        # Handle custom properties
        if is_custom_property(value_str):
            return f"({value_str})"
        
        # Handle arbitrary values
        if is_arbitrary_value(value_str) or value_str not in SHADOW_SIZES:
            return f"[{value_str}]"
        
        return value_str
    
    def _build_class(
        self,
        value: Optional[TailwindValue] = None  # Optional value to use
    ) -> str:  # The complete CSS class string
        """Build the shadow class string."""
        if value is not None:
            self._value = self._format_value(value)
        
        # If no value set, return just "shadow" (defaults to shadow-md in v4)
        if not hasattr(self, '_value') or self._value is None:
            return self.prefix
        
        # For standard sizes and arbitrary values
        base_class = f"{self.prefix}-{self._value}"
        
        # Apply modifiers
        if self._modifiers:
            return ":".join(self._modifiers + [base_class])
        
        return base_class

In [ ]:
#| export
class ShadowFactory(BaseFactory):
    """Factory for shadow utilities with size and color support."""
    
    def __init__(
        self,
        doc: Optional[str] = None  # Documentation
    ):
        """Initialize shadow factory with size properties."""
        doc = doc or "Box shadow utilities for adding depth to elements"
        super().__init__(doc)
        
        # Create properties for standard sizes
        self._2xs = ShadowUtility("2xs")
        self.xs = ShadowUtility("xs")
        self.sm = ShadowUtility("sm")
        self.md = ShadowUtility("md")
        self.lg = ShadowUtility("lg")
        self.xl = ShadowUtility("xl")
        self._2xl = ShadowUtility("2xl")
        self.none = ShadowUtility("none")
    
    def __call__(
        self,
        size: Optional[str] = None  # Shadow size or custom value
    ) -> ShadowUtility:  # A shadow utility instance
        """Create shadow utility with optional size."""
        return ShadowUtility(size)
    
    def __getattr__(
        self,
        name: str  # Attribute name
    ) -> ShadowUtility:  # A shadow utility instance
        """Handle size access (e.g., shadow._2xs)."""
        # Handle underscore prefix for sizes starting with numbers
        if name.startswith("_") and name[1:] + name[1:].endswith("xs") or name[1:] + name[1:].endswith("xl"):
            size = name[1:]
            if size in SHADOW_SIZES:
                return ShadowUtility(size)
        
        # Handle arbitrary values
        return ShadowUtility(name.replace("_", "-"))
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Factory information
        """Get information about the shadow factory."""
        return {
            'description': self._doc,
            'valid_inputs': {
                'sizes': list(SHADOW_SIZES.keys()),
                'usage_examples': [
                    'shadow() or shadow.md - default medium shadow',
                    'shadow.sm - small shadow',
                    'shadow.lg - large shadow',
                    'shadow.none - remove shadow',
                    'shadow("10px 10px 5px gray") - arbitrary shadow',
                    'shadow("--custom-shadow") - CSS custom property'
                ]
            },
            'options': {
                'default': 'shadow-md when no size specified',
                'supports_color': 'Use shadow_color factory for shadow colors'
            }
        }

In [ ]:
#| export
shadow = ShadowFactory("Box shadow utilities for adding depth to elements") # Create the shadow factory instance

### Shadow Color

Control the color of shadows using the standard color palette:

In [ ]:
#| export
shadow_color = ColoredFactory("shadow", "Shadow color utilities for customizing the color of box shadows") # Create shadow color factory using the existing ColoredFactory

### Test Shadow Utilities

Test the shadow size utilities:

In [ ]:
#| export
def test_effects_shadow_size_examples():
    """Test shadow size utilities."""
    # Test all named sizes
    assert str(shadow._2xs) == "shadow-2xs"
    assert str(shadow.xs) == "shadow-xs"
    assert str(shadow.sm) == "shadow-sm"
    assert str(shadow.md) == "shadow-md"
    assert str(shadow.lg) == "shadow-lg"
    assert str(shadow.xl) == "shadow-xl"
    assert str(shadow._2xl) == "shadow-2xl"
    assert str(shadow.none) == "shadow-none"
    
    # Test default shadow (no size)
    assert str(shadow()) == "shadow"
    
    # Test with size parameter
    assert str(shadow("sm")) == "shadow-sm"
    assert str(shadow("lg")) == "shadow-lg"
    assert str(shadow("none")) == "shadow-none"

# Run the tests
test_effects_shadow_size_examples()

In [ ]:
#| export
def test_effects_shadow_arbitrary_examples():
    """Test shadow utilities with arbitrary and custom values."""
    # Test arbitrary shadow values
    assert str(shadow("10px 10px 5px gray")) == "shadow-[10px 10px 5px gray]"
    assert str(shadow("0 4px 6px rgba(0,0,0,0.1)")) == "shadow-[0 4px 6px rgba(0,0,0,0.1)]"
    assert str(shadow("inset 0 2px 4px rgba(0,0,0,0.06)")) == "shadow-[inset 0 2px 4px rgba(0,0,0,0.06)]"
    
    # Test CSS custom properties
    assert str(shadow("--custom-shadow")) == "shadow-(--custom-shadow)"
    assert str(shadow("--elevation-high")) == "shadow-(--elevation-high)"

# Run the tests  
test_effects_shadow_arbitrary_examples()

### Test Shadow Color Utilities

Test the shadow color utilities:

In [ ]:
#| export
def test_effects_shadow_color_examples():
    """Test shadow color utilities with various color values."""
    # Test all 22 Tailwind color families
    assert str(shadow_color.red._500) == "shadow-red-500"
    assert str(shadow_color.orange._500) == "shadow-orange-500"
    assert str(shadow_color.amber._500) == "shadow-amber-500"
    assert str(shadow_color.yellow._500) == "shadow-yellow-500"
    assert str(shadow_color.lime._500) == "shadow-lime-500"
    assert str(shadow_color.green._500) == "shadow-green-500"
    assert str(shadow_color.emerald._500) == "shadow-emerald-500"
    assert str(shadow_color.teal._500) == "shadow-teal-500"
    assert str(shadow_color.cyan._500) == "shadow-cyan-500"
    assert str(shadow_color.sky._500) == "shadow-sky-500"
    assert str(shadow_color.blue._500) == "shadow-blue-500"
    assert str(shadow_color.indigo._500) == "shadow-indigo-500"
    assert str(shadow_color.violet._500) == "shadow-violet-500"
    assert str(shadow_color.purple._500) == "shadow-purple-500"
    assert str(shadow_color.fuchsia._500) == "shadow-fuchsia-500"
    assert str(shadow_color.pink._500) == "shadow-pink-500"
    assert str(shadow_color.rose._500) == "shadow-rose-500"
    assert str(shadow_color.slate._500) == "shadow-slate-500"
    assert str(shadow_color.gray._500) == "shadow-gray-500"
    assert str(shadow_color.zinc._500) == "shadow-zinc-500"
    assert str(shadow_color.neutral._500) == "shadow-neutral-500"
    assert str(shadow_color.stone._500) == "shadow-stone-500"
    
    # Test different shades
    assert str(shadow_color.blue._300) == "shadow-blue-300"
    assert str(shadow_color.green._950) == "shadow-green-950"
    
    # Special colors
    assert str(shadow_color.transparent) == "shadow-transparent"
    assert str(shadow_color.black) == "shadow-black"
    assert str(shadow_color.white) == "shadow-white"
    assert str(shadow_color.current) == "shadow-current"
    assert str(shadow_color.inherit) == "shadow-inherit"

# Run the tests
test_effects_shadow_color_examples()

In [ ]:
#| export
def test_effects_shadow_color_arbitrary_examples():
    """Test shadow color utilities with arbitrary values."""
    # Test hex colors
    assert str(shadow_color("#ff0000")) == "shadow-[#ff0000]"
    assert str(shadow_color("#123456")) == "shadow-[#123456]"
    
    # Test RGB/RGBA colors
    assert str(shadow_color("rgb(255, 0, 0)")) == "shadow-[rgb(255, 0, 0)]"
    assert str(shadow_color("rgba(0, 0, 0, 0.5)")) == "shadow-[rgba(0, 0, 0, 0.5)]"
    
    # Test CSS custom properties
    assert str(shadow_color("--shadow-color-primary")) == "shadow-(--shadow-color-primary)"
    assert str(shadow_color("--brand-shadow")) == "shadow-(--brand-shadow)"

# Run the tests
test_effects_shadow_color_arbitrary_examples()

## Inset Shadow

Utilities for controlling inset box shadows. These shadows appear inside the element rather than outside.

### Inset Shadow Size

Define the available inset shadow sizes:

In [ ]:
#| export
# Inset shadow size values mapped to their CSS variable names
INSET_SHADOW_SIZES = {
    "2xs": "inset-shadow-2xs",   # var(--inset-shadow-2xs) - inset 0 1px rgb(0 0 0 / 0.05)
    "xs": "inset-shadow-xs",      # var(--inset-shadow-xs) - inset 0 1px 1px rgb(0 0 0 / 0.05)
    "sm": "inset-shadow-sm",      # var(--inset-shadow-sm) - inset 0 2px 4px rgb(0 0 0 / 0.05)
    "none": "inset-shadow-none"   # inset 0 0 #0000
}

In [ ]:
#| export
class InsetShadowUtility(BaseUtility):
    """Utility class for inset box shadows with size support."""
    
    def __init__(
        self,
        size: Optional[str] = None,  # Inset shadow size or custom value
    ):
        """Initialize inset shadow utility with optional size."""
        super().__init__("inset-shadow")
        if size is not None:
            self._value = self._format_value(size)
    
    def _format_value(
        self,
        value: TailwindValue  # The value to format (size or arbitrary)
    ) -> str:  # The formatted value string
        """Format inset shadow value according to Tailwind conventions."""
        value_str = str(value)
        
        # Check if it's a standard size
        if value_str in INSET_SHADOW_SIZES:
            # Return just the size part (e.g., "sm", "xs")
            return value_str
        
        # Handle custom properties
        if is_custom_property(value_str):
            return f"({value_str})"
        
        # Handle arbitrary values
        if is_arbitrary_value(value_str) or value_str not in INSET_SHADOW_SIZES:
            return f"[{value_str}]"
        
        return value_str
    
    def _build_class(
        self,
        value: Optional[TailwindValue] = None  # Optional value to use
    ) -> str:  # The complete CSS class string
        """Build the inset shadow class string."""
        if value is not None:
            self._value = self._format_value(value)
        
        # For inset shadows, we need to have a value
        if not hasattr(self, '_value') or self._value is None:
            # Default to 'sm' if no value provided
            self._value = "sm"
        
        # For standard sizes and arbitrary values
        base_class = f"{self.prefix}-{self._value}"
        
        # Apply modifiers
        if self._modifiers:
            return ":".join(self._modifiers + [base_class])
        
        return base_class

In [ ]:
#| export
class InsetShadowFactory(BaseFactory):
    """Factory for inset shadow utilities with size support."""
    
    def __init__(
        self,
        doc: Optional[str] = None  # Documentation
    ):
        """Initialize inset shadow factory with size properties."""
        doc = doc or "Inset box shadow utilities for adding inner depth to elements"
        super().__init__(doc)
        
        # Create properties for standard sizes
        self._2xs = InsetShadowUtility("2xs")
        self.xs = InsetShadowUtility("xs")
        self.sm = InsetShadowUtility("sm")
        self.none = InsetShadowUtility("none")
    
    def __call__(
        self,
        size: Optional[str] = None  # Inset shadow size or custom value
    ) -> InsetShadowUtility:  # An inset shadow utility instance
        """Create inset shadow utility with optional size."""
        return InsetShadowUtility(size)
    
    def __getattr__(
        self,
        name: str  # Attribute name
    ) -> InsetShadowUtility:  # An inset shadow utility instance
        """Handle size access (e.g., inset_shadow._2xs)."""
        # Handle underscore prefix for sizes starting with numbers
        if name.startswith("_") and name[1:] + name[1:].endswith("xs"):
            size = name[1:]
            if size in INSET_SHADOW_SIZES:
                return InsetShadowUtility(size)
        
        # Handle arbitrary values
        return InsetShadowUtility(name.replace("_", "-"))
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Factory information
        """Get information about the inset shadow factory."""
        return {
            'description': self._doc,
            'valid_inputs': {
                'sizes': list(INSET_SHADOW_SIZES.keys()),
                'usage_examples': [
                    'inset_shadow() or inset_shadow.sm - default small inset shadow',
                    'inset_shadow.xs - extra small inset shadow',
                    'inset_shadow._2xs - extra extra small inset shadow',
                    'inset_shadow.none - remove inset shadow',
                    'inset_shadow("inset 0 4px 6px rgba(0,0,0,0.1)") - arbitrary inset shadow',
                    'inset_shadow("--custom-inset-shadow") - CSS custom property'
                ]
            },
            'options': {
                'default': 'inset-shadow-sm when no size specified',
                'supports_color': 'Use inset_shadow_color factory for inset shadow colors'
            }
        }

In [ ]:
#| export
inset_shadow = InsetShadowFactory("Inset box shadow utilities for adding inner depth to elements") # Create the inset shadow factory instance

### Inset Shadow Color

Control the color of inset shadows using the standard color palette:

In [ ]:
#| export
inset_shadow_color = ColoredFactory("inset-shadow", "Inset shadow color utilities for customizing the color of inner box shadows") # Create inset shadow color factory using the existing ColoredFactory

### Test Inset Shadow Utilities

Test the inset shadow size utilities:

In [ ]:
#| export
def test_effects_inset_shadow_size_examples():
    """Test inset shadow size utilities."""
    # Test all named sizes
    assert str(inset_shadow._2xs) == "inset-shadow-2xs"
    assert str(inset_shadow.xs) == "inset-shadow-xs"
    assert str(inset_shadow.sm) == "inset-shadow-sm"
    assert str(inset_shadow.none) == "inset-shadow-none"
    
    # Test default inset shadow (defaults to sm)
    assert str(inset_shadow()) == "inset-shadow-sm"
    
    # Test with size parameter
    assert str(inset_shadow("xs")) == "inset-shadow-xs"
    assert str(inset_shadow("2xs")) == "inset-shadow-2xs"
    assert str(inset_shadow("none")) == "inset-shadow-none"

# Run the tests
test_effects_inset_shadow_size_examples()

In [ ]:
#| export
def test_effects_inset_shadow_arbitrary_examples():
    """Test inset shadow utilities with arbitrary and custom values."""
    # Test arbitrary inset shadow values
    assert str(inset_shadow("inset 0 2px 4px rgba(0,0,0,0.06)")) == "inset-shadow-[inset 0 2px 4px rgba(0,0,0,0.06)]"
    assert str(inset_shadow("inset 0 1px 3px rgba(0,0,0,0.12)")) == "inset-shadow-[inset 0 1px 3px rgba(0,0,0,0.12)]"
    
    # Test CSS custom properties
    assert str(inset_shadow("--custom-inset-shadow")) == "inset-shadow-(--custom-inset-shadow)"
    assert str(inset_shadow("--inner-glow")) == "inset-shadow-(--inner-glow)"

# Run the tests  
test_effects_inset_shadow_arbitrary_examples()

### Test Inset Shadow Color Utilities

Test the inset shadow color utilities:

In [ ]:
#| export
def test_effects_inset_shadow_color_examples():
    """Test inset shadow color utilities with various color values."""
    # Test standard color families
    assert str(inset_shadow_color.red._500) == "inset-shadow-red-500"
    assert str(inset_shadow_color.blue._300) == "inset-shadow-blue-300"
    assert str(inset_shadow_color.green._950) == "inset-shadow-green-950"
    assert str(inset_shadow_color.purple._600) == "inset-shadow-purple-600"

    # Test all 22 Tailwind color families
    assert str(inset_shadow_color.red._500) == "inset-shadow-red-500"
    assert str(inset_shadow_color.orange._500) == "inset-shadow-orange-500"
    assert str(inset_shadow_color.amber._500) == "inset-shadow-amber-500"
    assert str(inset_shadow_color.yellow._500) == "inset-shadow-yellow-500"
    assert str(inset_shadow_color.lime._500) == "inset-shadow-lime-500"
    assert str(inset_shadow_color.green._500) == "inset-shadow-green-500"
    assert str(inset_shadow_color.emerald._500) == "inset-shadow-emerald-500"
    assert str(inset_shadow_color.teal._500) == "inset-shadow-teal-500"
    assert str(inset_shadow_color.cyan._500) == "inset-shadow-cyan-500"
    assert str(inset_shadow_color.sky._500) == "inset-shadow-sky-500"
    assert str(inset_shadow_color.blue._500) == "inset-shadow-blue-500"
    assert str(inset_shadow_color.indigo._500) == "inset-shadow-indigo-500"
    assert str(inset_shadow_color.violet._500) == "inset-shadow-violet-500"
    assert str(inset_shadow_color.purple._500) == "inset-shadow-purple-500"
    assert str(inset_shadow_color.fuchsia._500) == "inset-shadow-fuchsia-500"
    assert str(inset_shadow_color.pink._500) == "inset-shadow-pink-500"
    assert str(inset_shadow_color.rose._500) == "inset-shadow-rose-500"
    assert str(inset_shadow_color.slate._500) == "inset-shadow-slate-500"
    assert str(inset_shadow_color.gray._500) == "inset-shadow-gray-500"
    assert str(inset_shadow_color.zinc._500) == "inset-shadow-zinc-500"
    assert str(inset_shadow_color.neutral._500) == "inset-shadow-neutral-500"
    assert str(inset_shadow_color.stone._500) == "inset-shadow-stone-500"
    
    # Special colors
    assert str(inset_shadow_color.transparent) == "inset-shadow-transparent"
    assert str(inset_shadow_color.black) == "inset-shadow-black"
    assert str(inset_shadow_color.white) == "inset-shadow-white"
    assert str(inset_shadow_color.current) == "inset-shadow-current"
    assert str(inset_shadow_color.inherit) == "inset-shadow-inherit"
    
    # Test with arbitrary colors
    assert str(inset_shadow_color("#ff0000")) == "inset-shadow-[#ff0000]"
    assert str(inset_shadow_color("rgba(0, 0, 0, 0.5)")) == "inset-shadow-[rgba(0, 0, 0, 0.5)]"
    
    # Test CSS custom properties
    assert str(inset_shadow_color("--inset-shadow-primary")) == "inset-shadow-(--inset-shadow-primary)"

# Run the tests
test_effects_inset_shadow_color_examples()

## Ring

Utilities for creating outline rings around elements. These are commonly used for focus states.

### Ring Width

Define the ring width utilities:

In [ ]:
#| export
class RingUtility(BaseUtility):
    """Utility class for ring (outline) shadows with width support."""
    
    def __init__(
        self,
        width: Optional[Union[int, str]] = None,  # Ring width in pixels or custom value
    ):
        """Initialize ring utility with optional width."""
        super().__init__("ring")
        if width is not None:
            self._value = self._format_value(width)
    
    def _format_value(
        self,
        value: TailwindValue  # The value to format (width or arbitrary)
    ) -> str:  # The formatted value string
        """Format ring value according to Tailwind conventions."""
        value_str = str(value)
        
        # Check if it's a number (width in pixels)
        if isinstance(value, int) or (isinstance(value_str, str) and value_str.isdigit()):
            return value_str
        
        # Handle custom properties
        if is_custom_property(value_str):
            return f"({value_str})"
        
        # Handle arbitrary values
        if is_arbitrary_value(value_str):
            return f"[{value_str}]"
        
        return value_str
    
    def _build_class(
        self,
        value: Optional[TailwindValue] = None  # Optional value to use
    ) -> str:  # The complete CSS class string
        """Build the ring class string."""
        if value is not None:
            self._value = self._format_value(value)
        
        # If no value set, return just "ring" (defaults to 1px width)
        if not hasattr(self, '_value') or self._value is None:
            return self.prefix
        
        # For width values and arbitrary values
        base_class = f"{self.prefix}-{self._value}"
        
        # Apply modifiers
        if self._modifiers:
            return ":".join(self._modifiers + [base_class])
        
        return base_class

In [ ]:
#| export
class RingFactory(BaseFactory):
    """Factory for ring utilities with width support."""
    
    def __init__(
        self,
        doc: Optional[str] = None  # Documentation
    ):
        """Initialize ring factory."""
        doc = doc or "Ring (outline) utilities for adding focus rings and outlines to elements"
        super().__init__(doc)
        
        # Create properties for common widths
        self._0 = RingUtility(0)
        self._1 = RingUtility(1)
        self._2 = RingUtility(2)
        self._4 = RingUtility(4)
        self._8 = RingUtility(8)
    
    def __call__(
        self,
        width: Optional[Union[int, str]] = None  # Ring width or custom value
    ) -> RingUtility:  # A ring utility instance
        """Create ring utility with optional width."""
        return RingUtility(width)
    
    def __getattr__(
        self,
        name: str  # Attribute name
    ) -> RingUtility:  # A ring utility instance
        """Handle width access (e.g., ring._0, ring._2)."""
        # Handle underscore prefix for numeric widths
        if name.startswith("_") and name[1:].isdigit():
            width = int(name[1:])
            return RingUtility(width)
        
        # Handle arbitrary values
        return RingUtility(name.replace("_", "-"))
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Factory information
        """Get information about the ring factory."""
        return {
            'description': self._doc,
            'valid_inputs': {
                'widths': 'Any numeric value (0, 1, 2, 4, 8, etc.)',
                'usage_examples': [
                    'ring() - default 1px ring',
                    'ring._0 or ring(0) - remove ring',
                    'ring._2 or ring(2) - 2px ring',
                    'ring._4 or ring(4) - 4px ring',
                    'ring("3px") - arbitrary width',
                    'ring("--ring-width") - CSS custom property'
                ]
            },
            'options': {
                'default': 'ring (1px) when no width specified',
                'supports_color': 'Use ring_color factory for ring colors'
            }
        }

In [ ]:
#| export
ring = RingFactory("Ring (outline) utilities for adding focus rings and outlines to elements") # Create the ring factory instance

### Ring Color

Control the color of rings using the standard color palette:

In [ ]:
#| export
ring_color = ColoredFactory("ring", "Ring color utilities for customizing the color of focus rings and outlines") # Create ring color factory using the existing ColoredFactory

### Test Ring Utilities

Test the ring width utilities:

In [ ]:
#| export
def test_effects_ring_width_examples():
    """Test ring width utilities."""
    # Test default ring
    assert str(ring()) == "ring"
    
    # Test common widths
    assert str(ring._0) == "ring-0"
    assert str(ring._1) == "ring-1"
    assert str(ring._2) == "ring-2"
    assert str(ring._4) == "ring-4"
    assert str(ring._8) == "ring-8"
    
    # Test with width parameter
    assert str(ring(0)) == "ring-0"
    assert str(ring(2)) == "ring-2"
    assert str(ring(4)) == "ring-4"
    assert str(ring(16)) == "ring-16"
    
    # Test arbitrary values
    assert str(ring("3px")) == "ring-[3px]"
    assert str(ring("0.5rem")) == "ring-[0.5rem]"
    
    # Test CSS custom properties
    assert str(ring("--ring-width")) == "ring-(--ring-width)"
    assert str(ring("--focus-ring")) == "ring-(--focus-ring)"

# Run the tests
test_effects_ring_width_examples()

### Test Ring Color Utilities

Test the ring color utilities:

In [ ]:
#| export
def test_effects_ring_color_examples():
    """Test ring color utilities with various color values."""
    # Test standard color families
    assert str(ring_color.red._500) == "ring-red-500"
    assert str(ring_color.blue._300) == "ring-blue-300"
    assert str(ring_color.green._950) == "ring-green-950"
    assert str(ring_color.purple._600) == "ring-purple-600"

    # Test all 22 Tailwind color families
    assert str(ring_color.red._500) == "ring-red-500"
    assert str(ring_color.orange._500) == "ring-orange-500"
    assert str(ring_color.amber._500) == "ring-amber-500"
    assert str(ring_color.yellow._500) == "ring-yellow-500"
    assert str(ring_color.lime._500) == "ring-lime-500"
    assert str(ring_color.green._500) == "ring-green-500"
    assert str(ring_color.emerald._500) == "ring-emerald-500"
    assert str(ring_color.teal._500) == "ring-teal-500"
    assert str(ring_color.cyan._500) == "ring-cyan-500"
    assert str(ring_color.sky._500) == "ring-sky-500"
    assert str(ring_color.blue._500) == "ring-blue-500"
    assert str(ring_color.indigo._500) == "ring-indigo-500"
    assert str(ring_color.violet._500) == "ring-violet-500"
    assert str(ring_color.purple._500) == "ring-purple-500"
    assert str(ring_color.fuchsia._500) == "ring-fuchsia-500"
    assert str(ring_color.pink._500) == "ring-pink-500"
    assert str(ring_color.rose._500) == "ring-rose-500"
    assert str(ring_color.slate._500) == "ring-slate-500"
    assert str(ring_color.gray._500) == "ring-gray-500"
    assert str(ring_color.zinc._500) == "ring-zinc-500"
    assert str(ring_color.neutral._500) == "ring-neutral-500"
    assert str(ring_color.stone._500) == "ring-stone-500"
    
    # Special colors
    assert str(ring_color.transparent) == "ring-transparent"
    assert str(ring_color.black) == "ring-black"
    assert str(ring_color.white) == "ring-white"
    assert str(ring_color.current) == "ring-current"
    assert str(ring_color.inherit) == "ring-inherit"
    
    # Test with arbitrary colors
    assert str(ring_color("#ff0000")) == "ring-[#ff0000]"
    assert str(ring_color("rgba(0, 0, 255, 0.5)")) == "ring-[rgba(0, 0, 255, 0.5)]"
    
    # Test CSS custom properties
    assert str(ring_color("--ring-primary")) == "ring-(--ring-primary)"

# Run the tests
test_effects_ring_color_examples()

## Inset Ring

Utilities for creating inset (inner) outline rings. These appear inside the element rather than outside.

### Inset Ring Width

Define the inset ring width utilities:

In [ ]:
#| export
class InsetRingUtility(BaseUtility):
    """Utility class for inset ring (outline) shadows with width support."""
    
    def __init__(
        self,
        width: Optional[Union[int, str]] = None,  # Inset ring width in pixels or custom value
    ):
        """Initialize inset ring utility with optional width."""
        super().__init__("inset-ring")
        if width is not None:
            self._value = self._format_value(width)
    
    def _format_value(
        self,
        value: TailwindValue  # The value to format (width or arbitrary)
    ) -> str:  # The formatted value string
        """Format inset ring value according to Tailwind conventions."""
        value_str = str(value)
        
        # Check if it's a number (width in pixels)
        if isinstance(value, int) or (isinstance(value_str, str) and value_str.isdigit()):
            return value_str
        
        # Handle custom properties
        if is_custom_property(value_str):
            return f"({value_str})"
        
        # Handle arbitrary values
        if is_arbitrary_value(value_str):
            return f"[{value_str}]"
        
        return value_str
    
    def _build_class(
        self,
        value: Optional[TailwindValue] = None  # Optional value to use
    ) -> str:  # The complete CSS class string
        """Build the inset ring class string."""
        if value is not None:
            self._value = self._format_value(value)
        
        # If no value set, return just "inset-ring" (defaults to 1px width)
        if not hasattr(self, '_value') or self._value is None:
            return self.prefix
        
        # For width values and arbitrary values
        base_class = f"{self.prefix}-{self._value}"
        
        # Apply modifiers
        if self._modifiers:
            return ":".join(self._modifiers + [base_class])
        
        return base_class

In [ ]:
#| export
class InsetRingFactory(BaseFactory):
    """Factory for inset ring utilities with width support."""
    
    def __init__(
        self,
        doc: Optional[str] = None  # Documentation
    ):
        """Initialize inset ring factory."""
        doc = doc or "Inset ring (inner outline) utilities for adding inner focus rings and outlines to elements"
        super().__init__(doc)
        
        # Create properties for common widths
        self._0 = InsetRingUtility(0)
        self._1 = InsetRingUtility(1)
        self._2 = InsetRingUtility(2)
        self._4 = InsetRingUtility(4)
        self._8 = InsetRingUtility(8)
    
    def __call__(
        self,
        width: Optional[Union[int, str]] = None  # Inset ring width or custom value
    ) -> InsetRingUtility:  # An inset ring utility instance
        """Create inset ring utility with optional width."""
        return InsetRingUtility(width)
    
    def __getattr__(
        self,
        name: str  # Attribute name
    ) -> InsetRingUtility:  # An inset ring utility instance
        """Handle width access (e.g., inset_ring._0, inset_ring._2)."""
        # Handle underscore prefix for numeric widths
        if name.startswith("_") and name[1:].isdigit():
            width = int(name[1:])
            return InsetRingUtility(width)
        
        # Handle arbitrary values
        return InsetRingUtility(name.replace("_", "-"))
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Factory information
        """Get information about the inset ring factory."""
        return {
            'description': self._doc,
            'valid_inputs': {
                'widths': 'Any numeric value (0, 1, 2, 4, 8, etc.)',
                'usage_examples': [
                    'inset_ring() - default 1px inset ring',
                    'inset_ring._0 or inset_ring(0) - remove inset ring',
                    'inset_ring._2 or inset_ring(2) - 2px inset ring',
                    'inset_ring._4 or inset_ring(4) - 4px inset ring',
                    'inset_ring("3px") - arbitrary width',
                    'inset_ring("--inset-ring-width") - CSS custom property'
                ]
            },
            'options': {
                'default': 'inset-ring (1px) when no width specified',
                'supports_color': 'Use inset_ring_color factory for inset ring colors'
            }
        }

In [ ]:
#| export
inset_ring = InsetRingFactory("Inset ring (inner outline) utilities for adding inner focus rings and outlines to elements") # Create the inset ring factory instance

### Inset Ring Color

Control the color of inset rings using the standard color palette:

In [ ]:
#| export
inset_ring_color = ColoredFactory("inset-ring", "Inset ring color utilities for customizing the color of inner focus rings and outlines") # Create inset ring color factory using the existing ColoredFactory

### Test Inset Ring Utilities

Test the inset ring width utilities:

In [ ]:
#| export
def test_effects_inset_ring_width_examples():
    """Test inset ring width utilities."""
    # Test default inset ring
    assert str(inset_ring()) == "inset-ring"
    
    # Test common widths
    assert str(inset_ring._0) == "inset-ring-0"
    assert str(inset_ring._1) == "inset-ring-1"
    assert str(inset_ring._2) == "inset-ring-2"
    assert str(inset_ring._4) == "inset-ring-4"
    assert str(inset_ring._8) == "inset-ring-8"
    
    # Test with width parameter
    assert str(inset_ring(0)) == "inset-ring-0"
    assert str(inset_ring(2)) == "inset-ring-2"
    assert str(inset_ring(4)) == "inset-ring-4"
    assert str(inset_ring(16)) == "inset-ring-16"
    
    # Test arbitrary values
    assert str(inset_ring("3px")) == "inset-ring-[3px]"
    assert str(inset_ring("0.25rem")) == "inset-ring-[0.25rem]"
    
    # Test CSS custom properties
    assert str(inset_ring("--inset-ring-width")) == "inset-ring-(--inset-ring-width)"
    assert str(inset_ring("--inner-focus-ring")) == "inset-ring-(--inner-focus-ring)"

# Run the tests
test_effects_inset_ring_width_examples()

### Test Inset Ring Color Utilities

Test the inset ring color utilities:

In [ ]:
#| export
def test_effects_inset_ring_color_examples():
    """Test inset ring color utilities with various color values."""
    # Test standard color families
    assert str(inset_ring_color.red._500) == "inset-ring-red-500"
    assert str(inset_ring_color.blue._300) == "inset-ring-blue-300"
    assert str(inset_ring_color.green._950) == "inset-ring-green-950"
    assert str(inset_ring_color.purple._600) == "inset-ring-purple-600"

    # Test all 22 Tailwind color families
    assert str(inset_ring_color.red._500) == "inset-ring-red-500"
    assert str(inset_ring_color.orange._500) == "inset-ring-orange-500"
    assert str(inset_ring_color.amber._500) == "inset-ring-amber-500"
    assert str(inset_ring_color.yellow._500) == "inset-ring-yellow-500"
    assert str(inset_ring_color.lime._500) == "inset-ring-lime-500"
    assert str(inset_ring_color.green._500) == "inset-ring-green-500"
    assert str(inset_ring_color.emerald._500) == "inset-ring-emerald-500"
    assert str(inset_ring_color.teal._500) == "inset-ring-teal-500"
    assert str(inset_ring_color.cyan._500) == "inset-ring-cyan-500"
    assert str(inset_ring_color.sky._500) == "inset-ring-sky-500"
    assert str(inset_ring_color.blue._500) == "inset-ring-blue-500"
    assert str(inset_ring_color.indigo._500) == "inset-ring-indigo-500"
    assert str(inset_ring_color.violet._500) == "inset-ring-violet-500"
    assert str(inset_ring_color.purple._500) == "inset-ring-purple-500"
    assert str(inset_ring_color.fuchsia._500) == "inset-ring-fuchsia-500"
    assert str(inset_ring_color.pink._500) == "inset-ring-pink-500"
    assert str(inset_ring_color.rose._500) == "inset-ring-rose-500"
    assert str(inset_ring_color.slate._500) == "inset-ring-slate-500"
    assert str(inset_ring_color.gray._500) == "inset-ring-gray-500"
    assert str(inset_ring_color.zinc._500) == "inset-ring-zinc-500"
    assert str(inset_ring_color.neutral._500) == "inset-ring-neutral-500"
    assert str(inset_ring_color.stone._500) == "inset-ring-stone-500"
    
    # Special colors
    assert str(inset_ring_color.transparent) == "inset-ring-transparent"
    assert str(inset_ring_color.black) == "inset-ring-black"
    assert str(inset_ring_color.white) == "inset-ring-white"
    assert str(inset_ring_color.current) == "inset-ring-current"
    assert str(inset_ring_color.inherit) == "inset-ring-inherit"
    
    # Test with arbitrary colors
    assert str(inset_ring_color("#0000ff")) == "inset-ring-[#0000ff]"
    assert str(inset_ring_color("rgba(255, 0, 0, 0.8)")) == "inset-ring-[rgba(255, 0, 0, 0.8)]"
    
    # Test CSS custom properties
    assert str(inset_ring_color("--inset-ring-primary")) == "inset-ring-(--inset-ring-primary)"

# Run the tests
test_effects_inset_ring_color_examples()

## Text Shadow

Utilities for controlling the shadow of a text element. Text shadows in Tailwind CSS v4 support both shadow size and shadow color customization.

### Text Shadow Size

Define the available text shadow sizes:

In [ ]:
#| export
# Text shadow size values mapped to their CSS variable names
TEXT_SHADOW_SIZES = {
    "2xs": "text-shadow-2xs",   # var(--text-shadow-2xs) - 0px 1px 0px rgb(0 0 0 / 0.15)
    "xs": "text-shadow-xs",     # var(--text-shadow-xs) - 0px 1px 1px rgb(0 0 0 / 0.2)
    "sm": "text-shadow-sm",     # var(--text-shadow-sm) - 0px 1px 0px rgb(0 0 0 / 0.075), 0px 1px 1px rgb(0 0 0 / 0.075), 0px 2px 2px rgb(0 0 0 / 0.075)
    "md": "text-shadow-md",     # var(--text-shadow-md) - 0px 1px 1px rgb(0 0 0 / 0.1), 0px 1px 2px rgb(0 0 0 / 0.1), 0px 2px 4px rgb(0 0 0 / 0.1)
    "lg": "text-shadow-lg",     # var(--text-shadow-lg) - 0px 1px 2px rgb(0 0 0 / 0.1), 0px 3px 2px rgb(0 0 0 / 0.1), 0px 4px 8px rgb(0 0 0 / 0.1)
    "none": "text-shadow-none"  # none
}

In [ ]:
#| export
class TextShadowUtility(BaseUtility):
    """Utility class for text shadows with size support."""
    
    def __init__(
        self,
        size: Optional[str] = None,  # Text shadow size or custom value
    ):
        """Initialize text shadow utility with optional size."""
        super().__init__("text-shadow")
        if size is not None:
            self._value = self._format_value(size)
    
    def _format_value(
        self,
        value: TailwindValue  # The value to format (size or arbitrary)
    ) -> str:  # The formatted value string
        """Format text shadow value according to Tailwind conventions."""
        value_str = str(value)
        
        # Check if it's a standard size
        if value_str in TEXT_SHADOW_SIZES:
            # Return just the size part (e.g., "sm", "lg")
            return value_str
        
        # Handle custom properties
        if is_custom_property(value_str):
            return f"({value_str})"
        
        # Handle arbitrary values
        if is_arbitrary_value(value_str) or value_str not in TEXT_SHADOW_SIZES:
            return f"[{value_str}]"
        
        return value_str
    
    def _build_class(
        self,
        value: Optional[TailwindValue] = None  # Optional value to use
    ) -> str:  # The complete CSS class string
        """Build the text shadow class string."""
        if value is not None:
            self._value = self._format_value(value)
        
        # If no value set, return just "text-shadow" (defaults to text-shadow-md in v4)
        if not hasattr(self, '_value') or self._value is None:
            return self.prefix
        
        # For standard sizes and arbitrary values
        base_class = f"{self.prefix}-{self._value}"
        
        # Apply modifiers
        if self._modifiers:
            return ":".join(self._modifiers + [base_class])
        
        return base_class

In [ ]:
#| export
class TextShadowFactory(BaseFactory):
    """Factory for text shadow utilities with size support."""
    
    def __init__(
        self,
        doc: Optional[str] = None  # Documentation
    ):
        """Initialize text shadow factory with size properties."""
        doc = doc or "Text shadow utilities for adding shadows to text"
        super().__init__(doc)
        
        # Create properties for standard sizes
        self._2xs = TextShadowUtility("2xs")
        self.xs = TextShadowUtility("xs")
        self.sm = TextShadowUtility("sm")
        self.md = TextShadowUtility("md")
        self.lg = TextShadowUtility("lg")
        self.none = TextShadowUtility("none")
    
    def __call__(
        self,
        size: Optional[str] = None  # Text shadow size or custom value
    ) -> TextShadowUtility:  # A text shadow utility instance
        """Create text shadow utility with optional size."""
        return TextShadowUtility(size)
    
    def __getattr__(
        self,
        name: str  # Attribute name
    ) -> TextShadowUtility:  # A text shadow utility instance
        """Handle size access (e.g., text_shadow._2xs)."""
        # Handle underscore prefix for sizes starting with numbers
        if name.startswith("_") and (name[1:] + name[1:].endswith("xs") or name == "_2xs"):
            size = name[1:]
            if size in TEXT_SHADOW_SIZES:
                return TextShadowUtility(size)
        
        # Handle arbitrary values
        return TextShadowUtility(name.replace("_", "-"))
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Factory information
        """Get information about the text shadow factory."""
        return {
            'description': self._doc,
            'valid_inputs': {
                'sizes': list(TEXT_SHADOW_SIZES.keys()),
                'usage_examples': [
                    'text_shadow() or text_shadow.md - default medium text shadow',
                    'text_shadow.sm - small text shadow',
                    'text_shadow.lg - large text shadow',
                    'text_shadow.none - remove text shadow',
                    'text_shadow("2px 2px 4px rgba(0,0,0,0.5)") - arbitrary text shadow',
                    'text_shadow("--custom-text-shadow") - CSS custom property'
                ]
            },
            'options': {
                'default': 'text-shadow-md when no size specified',
                'supports_color': 'Use text_shadow_color factory for text shadow colors'
            }
        }

In [ ]:
#| export
text_shadow = TextShadowFactory("Text shadow utilities for adding shadows to text") # Create the text shadow factory instance

### Text Shadow Color

Control the color of text shadows using the standard color palette:

In [ ]:
#| export
text_shadow_color = ColoredFactory("text-shadow", "Text shadow color utilities for customizing the color of text shadows") # Create text shadow color factory using the existing ColoredFactory

### Test Text Shadow Utilities

Test the text shadow size utilities:

In [ ]:
#| export
def test_effects_text_shadow_size_examples():
    """Test text shadow size utilities."""
    # Test all named sizes
    assert str(text_shadow._2xs) == "text-shadow-2xs"
    assert str(text_shadow.xs) == "text-shadow-xs"
    assert str(text_shadow.sm) == "text-shadow-sm"
    assert str(text_shadow.md) == "text-shadow-md"
    assert str(text_shadow.lg) == "text-shadow-lg"
    assert str(text_shadow.none) == "text-shadow-none"
    
    # Test default text shadow (no size)
    assert str(text_shadow()) == "text-shadow"
    
    # Test with size parameter
    assert str(text_shadow("sm")) == "text-shadow-sm"
    assert str(text_shadow("lg")) == "text-shadow-lg"
    assert str(text_shadow("none")) == "text-shadow-none"

# Run the tests
test_effects_text_shadow_size_examples()

In [ ]:
#| export
def test_effects_text_shadow_arbitrary_examples():
    """Test text shadow utilities with arbitrary and custom values."""
    # Test arbitrary text shadow values
    assert str(text_shadow("2px 2px 4px rgba(0,0,0,0.5)")) == "text-shadow-[2px 2px 4px rgba(0,0,0,0.5)]"
    assert str(text_shadow("1px 1px 2px black")) == "text-shadow-[1px 1px 2px black]"
    assert str(text_shadow("0 0 10px #ff0000")) == "text-shadow-[0 0 10px #ff0000]"
    
    # Test CSS custom properties
    assert str(text_shadow("--custom-text-shadow")) == "text-shadow-(--custom-text-shadow)"
    assert str(text_shadow("--heading-shadow")) == "text-shadow-(--heading-shadow)"

# Run the tests  
test_effects_text_shadow_arbitrary_examples()

### Test Text Shadow Color Utilities

Test the text shadow color utilities:

In [ ]:
#| export
def test_effects_text_shadow_color_examples():
    """Test text shadow color utilities with various color values."""
    # Test all 22 Tailwind color families
    assert str(text_shadow_color.red._500) == "text-shadow-red-500"
    assert str(text_shadow_color.orange._500) == "text-shadow-orange-500"
    assert str(text_shadow_color.amber._500) == "text-shadow-amber-500"
    assert str(text_shadow_color.yellow._500) == "text-shadow-yellow-500"
    assert str(text_shadow_color.lime._500) == "text-shadow-lime-500"
    assert str(text_shadow_color.green._500) == "text-shadow-green-500"
    assert str(text_shadow_color.emerald._500) == "text-shadow-emerald-500"
    assert str(text_shadow_color.teal._500) == "text-shadow-teal-500"
    assert str(text_shadow_color.cyan._500) == "text-shadow-cyan-500"
    assert str(text_shadow_color.sky._500) == "text-shadow-sky-500"
    assert str(text_shadow_color.blue._500) == "text-shadow-blue-500"
    assert str(text_shadow_color.indigo._500) == "text-shadow-indigo-500"
    assert str(text_shadow_color.violet._500) == "text-shadow-violet-500"
    assert str(text_shadow_color.purple._500) == "text-shadow-purple-500"
    assert str(text_shadow_color.fuchsia._500) == "text-shadow-fuchsia-500"
    assert str(text_shadow_color.pink._500) == "text-shadow-pink-500"
    assert str(text_shadow_color.rose._500) == "text-shadow-rose-500"
    assert str(text_shadow_color.slate._500) == "text-shadow-slate-500"
    assert str(text_shadow_color.gray._500) == "text-shadow-gray-500"
    assert str(text_shadow_color.zinc._500) == "text-shadow-zinc-500"
    assert str(text_shadow_color.neutral._500) == "text-shadow-neutral-500"
    assert str(text_shadow_color.stone._500) == "text-shadow-stone-500"
    
    # Test different shades
    assert str(text_shadow_color.blue._300) == "text-shadow-blue-300"
    assert str(text_shadow_color.green._950) == "text-shadow-green-950"
    
    # Special colors
    assert str(text_shadow_color.transparent) == "text-shadow-transparent"
    assert str(text_shadow_color.black) == "text-shadow-black"
    assert str(text_shadow_color.white) == "text-shadow-white"
    assert str(text_shadow_color.current) == "text-shadow-current"
    assert str(text_shadow_color.inherit) == "text-shadow-inherit"

# Run the tests
test_effects_text_shadow_color_examples()

## Opacity

Utilities for controlling the opacity of an element. Opacity values range from 0 (fully transparent) to 100 (fully opaque).

In [ ]:
#| export
class OpacityUtility(BaseUtility):
    """Utility class for opacity with percentage support."""
    
    def __init__(
        self,
        value: Optional[Union[int, str]] = None,  # Opacity value (0-100) or custom value
    ):
        """Initialize opacity utility with optional value."""
        super().__init__("opacity")
        if value is not None:
            self._value = self._format_value(value)
    
    def _format_value(
        self,
        value: TailwindValue  # The value to format (percentage or arbitrary)
    ) -> str:  # The formatted value string
        """Format opacity value according to Tailwind conventions."""
        # Check if it's a number (0-100)
        if isinstance(value, int) or (isinstance(value, str) and value.isdigit()):
            return str(value)
        
        value_str = str(value)
        
        # Handle custom properties
        if is_custom_property(value_str):
            return f"({value_str})"
        
        # Handle arbitrary values (like decimal values)
        if is_arbitrary_value(value_str) or '.' in value_str:
            return f"[{value_str}]"
        
        return value_str
    
    def _build_class(
        self,
        value: Optional[TailwindValue] = None  # Optional value to use
    ) -> str:  # The complete CSS class string
        """Build the opacity class string."""
        if value is not None:
            self._value = self._format_value(value)
        
        # Opacity requires a value
        if not hasattr(self, '_value') or self._value is None:
            raise ValueError("Opacity requires a value (0-100)")
        
        # Build the class
        base_class = f"{self.prefix}-{self._value}"
        
        # Apply modifiers
        if self._modifiers:
            return ":".join(self._modifiers + [base_class])
        
        return base_class

In [ ]:
#| export
class OpacityFactory(BaseFactory):
    """Factory for opacity utilities."""
    
    def __init__(
        self,
        doc: Optional[str] = None  # Documentation
    ):
        """Initialize opacity factory with common opacity properties."""
        doc = doc or "Opacity utilities for controlling element transparency"
        super().__init__(doc)
        
        # Create properties for common opacity values
        self._0 = OpacityUtility(0)
        self._5 = OpacityUtility(5)
        self._10 = OpacityUtility(10)
        self._15 = OpacityUtility(15)
        self._20 = OpacityUtility(20)
        self._25 = OpacityUtility(25)
        self._30 = OpacityUtility(30)
        self._35 = OpacityUtility(35)
        self._40 = OpacityUtility(40)
        self._45 = OpacityUtility(45)
        self._50 = OpacityUtility(50)
        self._55 = OpacityUtility(55)
        self._60 = OpacityUtility(60)
        self._65 = OpacityUtility(65)
        self._70 = OpacityUtility(70)
        self._75 = OpacityUtility(75)
        self._80 = OpacityUtility(80)
        self._85 = OpacityUtility(85)
        self._90 = OpacityUtility(90)
        self._95 = OpacityUtility(95)
        self._100 = OpacityUtility(100)
    
    def __call__(
        self,
        value: Union[int, str]  # Opacity value (0-100) or custom value
    ) -> OpacityUtility:  # An opacity utility instance
        """Create opacity utility with value."""
        return OpacityUtility(value)
    
    def __getattr__(
        self,
        name: str  # Attribute name
    ) -> OpacityUtility:  # An opacity utility instance
        """Handle opacity value access (e.g., opacity._50)."""
        # Handle underscore prefix for numeric values
        if name.startswith("_") and name[1:].isdigit():
            value = int(name[1:])
            if 0 <= value <= 100:
                return OpacityUtility(value)
        
        # Handle arbitrary values
        return OpacityUtility(name.replace("_", "-"))
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Factory information
        """Get information about the opacity factory."""
        return {
            'description': self._doc,
            'valid_inputs': {
                'percentages': 'Integer values from 0 to 100',
                'common_values': [0, 5, 10, 20, 25, 30, 40, 50, 60, 70, 75, 80, 90, 95, 100],
                'usage_examples': [
                    'opacity(50) or opacity._50 - 50% opacity',
                    'opacity._0 - fully transparent',
                    'opacity._100 - fully opaque',
                    'opacity("0.87") - arbitrary decimal value',
                    'opacity("--custom-opacity") - CSS custom property'
                ]
            },
            'options': {
                'range': '0 (transparent) to 100 (opaque)',
                'increments': 'Commonly in steps of 5 or 10'
            }
        }

In [ ]:
#| export
opacity = OpacityFactory("Opacity utilities for controlling element transparency") # Create the opacity factory instance

### Test Opacity Utilities

Test the opacity utilities:

In [ ]:
#| export
def test_effects_opacity_examples():
    """Test opacity utilities with various values."""
    # Test common opacity values
    assert str(opacity._0) == "opacity-0"
    assert str(opacity._5) == "opacity-5"
    assert str(opacity._10) == "opacity-10"
    assert str(opacity._20) == "opacity-20"
    assert str(opacity._25) == "opacity-25"
    assert str(opacity._30) == "opacity-30"
    assert str(opacity._40) == "opacity-40"
    assert str(opacity._50) == "opacity-50"
    assert str(opacity._60) == "opacity-60"
    assert str(opacity._70) == "opacity-70"
    assert str(opacity._75) == "opacity-75"
    assert str(opacity._80) == "opacity-80"
    assert str(opacity._90) == "opacity-90"
    assert str(opacity._95) == "opacity-95"
    assert str(opacity._100) == "opacity-100"
    
    # Test with value parameter
    assert str(opacity(0)) == "opacity-0"
    assert str(opacity(50)) == "opacity-50"
    assert str(opacity(100)) == "opacity-100"
    
    # Test arbitrary decimal values
    assert str(opacity("0.87")) == "opacity-[0.87]"
    assert str(opacity("0.5")) == "opacity-[0.5]"
    assert str(opacity("0.333")) == "opacity-[0.333]"
    
    # Test CSS custom properties
    assert str(opacity("--custom-opacity")) == "opacity-(--custom-opacity)"
    assert str(opacity("--fade-amount")) == "opacity-(--fade-amount)"

# Run the tests
test_effects_opacity_examples()

## Mix Blend Mode

Utilities for controlling how an element should blend with the background.

In [ ]:
#| export
mix_blend = SimpleFactory(
    {
        "normal": "mix-blend-normal",
        "multiply": "mix-blend-multiply",
        "screen": "mix-blend-screen",
        "overlay": "mix-blend-overlay",
        "darken": "mix-blend-darken",
        "lighten": "mix-blend-lighten",
        "color_dodge": "mix-blend-color-dodge",
        "color_burn": "mix-blend-color-burn",
        "hard_light": "mix-blend-hard-light",
        "soft_light": "mix-blend-soft-light",
        "difference": "mix-blend-difference",
        "exclusion": "mix-blend-exclusion",
        "hue": "mix-blend-hue",
        "saturation": "mix-blend-saturation",
        "color": "mix-blend-color",
        "luminosity": "mix-blend-luminosity",
        "plus_darker": "mix-blend-plus-darker",
        "plus_lighter": "mix-blend-plus-lighter"
    },
    "Mix blend mode utilities for controlling how elements blend with their background"
) # Mix blend mode factory

In [ ]:
#| export
def test_effects_mix_blend_examples():
    """Test mix blend mode utilities."""
    assert str(mix_blend.normal) == "mix-blend-normal"
    assert str(mix_blend.multiply) == "mix-blend-multiply"
    assert str(mix_blend.screen) == "mix-blend-screen"
    assert str(mix_blend.overlay) == "mix-blend-overlay"
    assert str(mix_blend.darken) == "mix-blend-darken"
    assert str(mix_blend.lighten) == "mix-blend-lighten"
    assert str(mix_blend.color_dodge) == "mix-blend-color-dodge"
    assert str(mix_blend.color_burn) == "mix-blend-color-burn"
    assert str(mix_blend.hard_light) == "mix-blend-hard-light"
    assert str(mix_blend.soft_light) == "mix-blend-soft-light"
    assert str(mix_blend.difference) == "mix-blend-difference"
    assert str(mix_blend.exclusion) == "mix-blend-exclusion"
    assert str(mix_blend.hue) == "mix-blend-hue"
    assert str(mix_blend.saturation) == "mix-blend-saturation"
    assert str(mix_blend.color) == "mix-blend-color"
    assert str(mix_blend.luminosity) == "mix-blend-luminosity"
    assert str(mix_blend.plus_darker) == "mix-blend-plus-darker"
    assert str(mix_blend.plus_lighter) == "mix-blend-plus-lighter"

# Run the tests
test_effects_mix_blend_examples()

## Background Blend Mode

Utilities for controlling how an element's background image should blend with its background color.

In [ ]:
#| export
bg_blend = SimpleFactory(
    {
        "normal": "bg-blend-normal",
        "multiply": "bg-blend-multiply",
        "screen": "bg-blend-screen",
        "overlay": "bg-blend-overlay",
        "darken": "bg-blend-darken",
        "lighten": "bg-blend-lighten",
        "color_dodge": "bg-blend-color-dodge",
        "color_burn": "bg-blend-color-burn",
        "hard_light": "bg-blend-hard-light",
        "soft_light": "bg-blend-soft-light",
        "difference": "bg-blend-difference",
        "exclusion": "bg-blend-exclusion",
        "hue": "bg-blend-hue",
        "saturation": "bg-blend-saturation",
        "color": "bg-blend-color",
        "luminosity": "bg-blend-luminosity"
    },
    "Background blend mode utilities for controlling how background images blend with background colors"
) # Background blend mode factory

In [ ]:
#| export
def test_effects_bg_blend_examples():
    """Test background blend mode utilities."""
    assert str(bg_blend.normal) == "bg-blend-normal"
    assert str(bg_blend.multiply) == "bg-blend-multiply"
    assert str(bg_blend.screen) == "bg-blend-screen"
    assert str(bg_blend.overlay) == "bg-blend-overlay"
    assert str(bg_blend.darken) == "bg-blend-darken"
    assert str(bg_blend.lighten) == "bg-blend-lighten"
    assert str(bg_blend.color_dodge) == "bg-blend-color-dodge"
    assert str(bg_blend.color_burn) == "bg-blend-color-burn"
    assert str(bg_blend.hard_light) == "bg-blend-hard-light"
    assert str(bg_blend.soft_light) == "bg-blend-soft-light"
    assert str(bg_blend.difference) == "bg-blend-difference"
    assert str(bg_blend.exclusion) == "bg-blend-exclusion"
    assert str(bg_blend.hue) == "bg-blend-hue"
    assert str(bg_blend.saturation) == "bg-blend-saturation"
    assert str(bg_blend.color) == "bg-blend-color"
    assert str(bg_blend.luminosity) == "bg-blend-luminosity"

# Run the tests
test_effects_bg_blend_examples()

## Mask Utilities

Utilities for controlling CSS mask properties. Masks allow you to hide portions of an element by masking it with an image or gradient.

### Mask Clip

Utilities for controlling the bounding box of an element's mask:

In [ ]:
#| export
mask_clip = SimpleFactory(
    {
        "border": "mask-clip-border",
        "padding": "mask-clip-padding",
        "content": "mask-clip-content",
        "fill": "mask-clip-fill",
        "stroke": "mask-clip-stroke",
        "view": "mask-clip-view",
        "no_clip": "mask-no-clip"
    },
    "Mask clip utilities for controlling the bounding box of an element's mask"
) # Mask clip factory

In [ ]:
#| export
def test_effects_mask_clip_examples():
    """Test mask clip utilities."""
    assert str(mask_clip.border) == "mask-clip-border"
    assert str(mask_clip.padding) == "mask-clip-padding"
    assert str(mask_clip.content) == "mask-clip-content"
    assert str(mask_clip.fill) == "mask-clip-fill"
    assert str(mask_clip.stroke) == "mask-clip-stroke"
    assert str(mask_clip.view) == "mask-clip-view"
    assert str(mask_clip.no_clip) == "mask-no-clip"

# Run the tests
test_effects_mask_clip_examples()

### Mask Composite

Utilities for controlling how multiple masks are combined together:

In [ ]:
#| export
mask_composite = SimpleFactory(
    {
        "add": "mask-add",
        "subtract": "mask-subtract",
        "intersect": "mask-intersect",
        "exclude": "mask-exclude"
    },
    "Mask composite utilities for controlling how multiple masks are combined"
) # Mask composite factory

In [ ]:
#| export
def test_effects_mask_composite_examples():
    """Test mask composite utilities."""
    assert str(mask_composite.add) == "mask-add"
    assert str(mask_composite.subtract) == "mask-subtract"
    assert str(mask_composite.intersect) == "mask-intersect"
    assert str(mask_composite.exclude) == "mask-exclude"

# Run the tests
test_effects_mask_composite_examples()

### Mask Image

Utilities for controlling an element's mask image. Masks allow you to create complex shapes and effects by using images or gradients to selectively show or hide parts of an element.

#### Basic Mask Image

Support for basic mask image utilities including mask-none and arbitrary values:

In [ ]:
#| export
class MaskImageUtility(BaseUtility):
    """Utility class for mask images."""
    
    def __init__(
        self,
        value: Optional[str] = None,  # Mask image value (none, custom property, or arbitrary)
    ):
        """Initialize mask image utility with optional value."""
        super().__init__("mask")
        if value is not None:
            self._value = self._format_value(value)
    
    def _format_value(
        self,
        value: TailwindValue  # The value to format
    ) -> str:  # The formatted value string
        """Format mask image value according to Tailwind conventions."""
        value_str = str(value)
        
        # Handle special value 'none'
        if value_str == "none":
            return "none"
        
        # Handle custom properties
        if is_custom_property(value_str):
            return f"({value_str})"
        
        # All other values are arbitrary
        return f"[{value_str}]"
    
    def _build_class(
        self,
        value: Optional[TailwindValue] = None  # Optional value to use
    ) -> str:  # The complete CSS class string
        """Build the mask image class string."""
        if value is not None:
            self._value = self._format_value(value)
        
        # If no value set, error - mask requires a value
        if not hasattr(self, '_value') or self._value is None:
            raise ValueError("Mask image requires a value")
        
        # Build the class
        base_class = f"{self.prefix}-{self._value}"
        
        # Apply modifiers
        if self._modifiers:
            return ":".join(self._modifiers + [base_class])
        
        return base_class

In [ ]:
#| export
class MaskImageFactory(BaseFactory):
    """Factory for basic mask image utilities."""
    
    def __init__(
        self,
        doc: Optional[str] = None  # Documentation
    ):
        """Initialize mask image factory."""
        doc = doc or "Basic mask image utilities for controlling element masks"
        super().__init__(doc)
        
        # Create mask-none utility
        self.none = MaskImageUtility("none")
    
    def __call__(
        self,
        value: str  # Mask image value (arbitrary or custom property)
    ) -> MaskImageUtility:  # A mask image utility instance
        """Create mask image utility with value."""
        return MaskImageUtility(value)
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Factory information
        """Get information about the mask image factory."""
        return {
            'description': self._doc,
            'valid_inputs': {
                'special_values': ['none'],
                'usage_examples': [
                    'mask.none - remove mask image',
                    'mask("url(#my-mask)") - SVG mask reference',
                    'mask("linear-gradient(black, transparent)") - gradient mask',
                    'mask("--custom-mask") - CSS custom property'
                ]
            },
            'options': {
                'additional_utilities': 'Use mask_linear, mask_radial, mask_conic for gradient masks'
            }
        }

mask = MaskImageFactory("Basic mask image utilities for controlling element masks") # Create the basic mask image factory instance

#### Linear Gradient Masks

Create linear gradient masks with angle support:

In [ ]:
#| export
class MaskLinearUtility(BaseUtility):
    """Utility class for linear gradient masks with angle support."""
    
    def __init__(
        self,
        angle: Optional[Union[int, str]] = None,  # Angle in degrees or custom value
        negative: bool = False  # Whether to negate the angle
    ):
        """Initialize linear gradient mask utility."""
        super().__init__("mask-linear")
        self.negative = negative
        if angle is not None:
            self._value = self._format_value(angle)
    
    def _format_value(
        self,
        value: TailwindValue  # The value to format
    ) -> str:  # The formatted value string
        """Format linear gradient mask value."""
        # Handle numeric angles
        if isinstance(value, int) or (isinstance(value, str) and value.isdigit()):
            return str(value)
        
        value_str = str(value)
        
        # Handle custom properties
        if is_custom_property(value_str):
            return f"({value_str})"
        
        # Handle arbitrary values
        return f"[{value_str}]"
    
    def _build_class(
        self,
        value: Optional[TailwindValue] = None  # Optional value to use
    ) -> str:  # The complete CSS class string
        """Build the linear gradient mask class string."""
        if value is not None:
            self._value = self._format_value(value)
        
        # Linear gradient mask requires an angle
        if not hasattr(self, '_value') or self._value is None:
            raise ValueError("Linear gradient mask requires an angle value")
        
        # Apply negative prefix for angles
        neg_prefix = "-" if self.negative and self._value.isdigit() else ""
        
        # Build the class
        base_class = f"{neg_prefix}{self.prefix}-{self._value}"
        
        # Apply modifiers
        if self._modifiers:
            return ":".join(self._modifiers + [base_class])
        
        return base_class

In [ ]:
#| export
class MaskLinearFactory(BaseFactory):
    """Factory for linear gradient mask utilities with angle support."""
    
    def __init__(
        self,
        doc: Optional[str] = None  # Documentation
    ):
        """Initialize linear gradient mask factory."""
        doc = doc or "Linear gradient mask utilities for creating gradient-based masks"
        super().__init__(doc)
        
        # Common angles
        self._0 = MaskLinearUtility(0)
        self._45 = MaskLinearUtility(45)
        self._90 = MaskLinearUtility(90)
        self._180 = MaskLinearUtility(180)
    
    def __call__(
        self,
        angle: Union[int, str],  # Angle in degrees or custom value
        negative: bool = False  # Whether to negate angle
    ) -> MaskLinearUtility:  # A linear gradient mask utility instance
        """Create linear gradient mask utility with angle."""
        return MaskLinearUtility(angle, negative)
    
    def __getattr__(
        self,
        name: str  # Attribute name (angle)
    ) -> MaskLinearUtility:  # A linear gradient mask utility instance
        """Handle angle access (e.g., mask_linear._45, mask_linear._90)."""
        # Handle underscore prefix for numeric angles
        if name.startswith("_") and name[1:].isdigit():
            angle = int(name[1:])
            return MaskLinearUtility(angle)
        
        # Handle negative angles (e.g., neg_45)
        if name.startswith("neg_") and name[4:].isdigit():
            angle = int(name[4:])
            return MaskLinearUtility(angle, negative=True)
        
        raise AttributeError(f"'{self.__class__.__name__}' object has no attribute '{name}'")
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Factory information
        """Get information about the linear gradient mask factory."""
        return {
            'description': self._doc,
            'valid_inputs': {
                'angles': 'Any numeric angle (0-360 degrees)',
                'usage_examples': [
                    'mask_linear(45) or mask_linear._45 - 45 degree gradient',
                    'mask_linear(90, negative=True) or mask_linear.neg_90 - negative 90 degree gradient',
                    'mask_linear._0 - horizontal gradient (0 degrees)',
                    'mask_linear._180 - horizontal gradient (180 degrees)',
                    'mask_linear("--gradient-angle") - CSS custom property'
                ]
            },
            'options': {
                'negative_angles': 'Use negative=True or neg_<angle> syntax',
                'from_to_positioning': 'Use mask_linear_from and mask_linear_to for gradient stops'
            }
        }

mask_linear = MaskLinearFactory("Linear gradient mask utilities for creating gradient-based masks") # Create the linear gradient mask factory instance

#### Directional Linear Gradient Masks

Create directional linear gradient masks with from/to positioning:

In [ ]:
#| export
class MaskDirectionalUtility(BaseUtility):
    """Utility class for directional mask gradients with from/to support."""
    
    def __init__(
        self,
        direction: str,  # Direction (t, r, b, l, x, y)
        position: str,  # Position type (from or to)
        value: Optional[Union[int, str]] = None,  # Value (number, percentage, color, etc.)
    ):
        """Initialize directional mask gradient utility."""
        super().__init__(f"mask-{direction}-{position}")
        self.direction = direction
        self.position = position
        if value is not None:
            self._value = self._format_value(value)
    
    def _format_value(
        self,
        value: TailwindValue  # The value to format
    ) -> str:  # The formatted value string
        """Format directional mask value according to type."""
        value_str = str(value)
        
        # Handle numeric values (spacing scale)
        if isinstance(value, int) or (isinstance(value_str, str) and value_str.isdigit()):
            return value_str
        
        # Handle percentages
        if value_str.endswith('%'):
            return value_str
        
        # Handle colors (check if it's a valid color)
        # For mask gradients, colors affect the mask opacity
        # We'll treat any non-numeric, non-percentage, non-custom-property as potential color
        if not is_custom_property(value_str) and not is_arbitrary_value(value_str):
            # It might be a color name like 'black', 'white', 'transparent'
            return value_str
        
        # Handle custom properties
        if is_custom_property(value_str):
            return f"({value_str})"
        
        # Handle arbitrary values
        return f"[{value_str}]"
    
    def _build_class(
        self,
        value: Optional[TailwindValue] = None  # Optional value to use
    ) -> str:  # The complete CSS class string
        """Build the directional mask class string."""
        if value is not None:
            self._value = self._format_value(value)
        
        # Directional mask requires a value
        if not hasattr(self, '_value') or self._value is None:
            raise ValueError(f"Directional mask {self.prefix} requires a value")
        
        # Build the class
        base_class = f"{self.prefix}-{self._value}"
        
        # Apply modifiers
        if self._modifiers:
            return ":".join(self._modifiers + [base_class])
        
        return base_class

In [ ]:
#| export
class MaskDirectionalFactory(BaseFactory):
    """Factory for directional mask gradient utilities."""
    
    def __init__(
        self,
        direction: str,  # Direction (t, r, b, l, x, y)
        position: str,  # Position type (from or to)
        doc: Optional[str] = None  # Documentation
    ):
        """Initialize directional mask factory."""
        self.direction = direction
        self.position = position
        
        direction_names = {
            't': 'top', 'r': 'right', 'b': 'bottom', 
            'l': 'left', 'x': 'horizontal', 'y': 'vertical'
        }
        dir_name = direction_names.get(direction, direction)
        
        doc = doc or f"Mask gradient {position} utilities for {dir_name} direction"
        super().__init__(doc)
    
    def __call__(
        self,
        value: Union[int, str]  # Value (number, percentage, color, etc.)
    ) -> MaskDirectionalUtility:  # A directional mask utility instance
        """Create directional mask utility with value."""
        return MaskDirectionalUtility(self.direction, self.position, value)
    
    def __getattr__(
        self,
        name: str  # Attribute name
    ) -> MaskDirectionalUtility:  # A directional mask utility instance
        """Handle value access (e.g., mask_t_from._50, mask_r_to.black)."""
        # Handle underscore prefix for numeric values
        if name.startswith("_") and name[1:].isdigit():
            value = int(name[1:])
            return MaskDirectionalUtility(self.direction, self.position, value)
        
        # Handle percentage values (e.g., _50p for 50%)
        if name.startswith("_") and name.endswith("p") and name[1:-1].isdigit():
            value = f"{name[1:-1]}%"
            return MaskDirectionalUtility(self.direction, self.position, value)
        
        # Handle named values (colors, etc.)
        return MaskDirectionalUtility(self.direction, self.position, name.replace("_", "-"))
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Factory information
        """Get information about this directional mask factory."""
        direction_names = {
            't': 'top', 'r': 'right', 'b': 'bottom', 
            'l': 'left', 'x': 'horizontal', 'y': 'vertical'
        }
        dir_name = direction_names.get(self.direction, self.direction)
        
        return {
            'description': self._doc,
            'direction': dir_name,
            'position_type': self.position,
            'valid_inputs': {
                'numeric': 'Spacing scale values (0-96)',
                'percentages': 'Percentage values (e.g., "50%", use _50p syntax)',
                'colors': 'Color names for mask opacity (black, white, transparent)',
                'custom_properties': 'CSS variables (e.g., "--mask-start")',
                'arbitrary': 'Any valid CSS value in brackets'
            },
            'usage_examples': [
                f'mask_{self.direction}_{self.position}(50) - numeric spacing value',
                f'mask_{self.direction}_{self.position}("50%") - percentage',
                f'mask_{self.direction}_{self.position}.black - full opacity',
                f'mask_{self.direction}_{self.position}.transparent - transparent'
            ]
        }

In [ ]:
#| export
# Create directional mask gradient factories
mask_t_from = MaskDirectionalFactory('t', 'from') # Top direction - from
mask_t_to = MaskDirectionalFactory('t', 'to') # Top direction - to

mask_r_from = MaskDirectionalFactory('r', 'from') # Right direction - from
mask_r_to = MaskDirectionalFactory('r', 'to') # Right direction - to

mask_b_from = MaskDirectionalFactory('b', 'from') # Bottom direction - from
mask_b_to = MaskDirectionalFactory('b', 'to') # Bottom direction - to

mask_l_from = MaskDirectionalFactory('l', 'from') # Left direction - from
mask_l_to = MaskDirectionalFactory('l', 'to') # Left direction - to

mask_x_from = MaskDirectionalFactory('x', 'from') # Horizontal (x) direction - from
mask_x_to = MaskDirectionalFactory('x', 'to') # Horizontal (x) direction - to

mask_y_from = MaskDirectionalFactory('y', 'from') # Vertical (y) direction - from
mask_y_to = MaskDirectionalFactory('y', 'to') # Vertical (y) direction - to

#### Radial Gradient Masks

Create radial gradient masks with shape, size, and position controls:

In [ ]:
#| export
class MaskRadialUtility(BaseUtility):
    """Utility class for radial gradient masks."""
    
    def __init__(
        self,
        value: Optional[str] = None,  # Arbitrary radial gradient value
    ):
        """Initialize radial gradient mask utility."""
        super().__init__("mask-radial")
        if value is not None:
            self._value = self._format_value(value)
    
    def _format_value(
        self,
        value: TailwindValue  # The value to format
    ) -> str:  # The formatted value string
        """Format radial gradient mask value."""
        value_str = str(value)
        
        # Handle custom properties
        if is_custom_property(value_str):
            return f"({value_str})"
        
        # All radial gradient values are arbitrary
        return f"[{value_str}]"
    
    def _build_class(
        self,
        value: Optional[TailwindValue] = None  # Optional value to use
    ) -> str:  # The complete CSS class string
        """Build the radial gradient mask class string."""
        if value is not None:
            self._value = self._format_value(value)
        
        # If no value, return just mask-radial
        if not hasattr(self, '_value') or self._value is None:
            return self.prefix
        
        # Build the class
        base_class = f"{self.prefix}-{self._value}"
        
        # Apply modifiers
        if self._modifiers:
            return ":".join(self._modifiers + [base_class])
        
        return base_class

In [ ]:
#| export
class MaskRadialFactory(BaseFactory):
    """Factory for radial gradient mask utilities."""
    
    def __init__(
        self,
        doc: Optional[str] = None  # Documentation
    ):
        """Initialize radial gradient mask factory."""
        doc = doc or "Radial gradient mask utilities for creating circular gradient-based masks"
        super().__init__(doc)
    
    def __call__(
        self,
        value: Optional[str] = None  # Arbitrary radial gradient value
    ) -> MaskRadialUtility:  # A radial gradient mask utility instance
        """Create radial gradient mask utility with optional value."""
        return MaskRadialUtility(value)
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Factory information
        """Get information about the radial gradient mask factory."""
        return {
            'description': self._doc,
            'valid_inputs': {
                'arbitrary': 'Any valid radial-gradient CSS value',
                'usage_examples': [
                    'mask_radial() - basic radial gradient mask',
                    'mask_radial("circle at center") - circular gradient at center',
                    'mask_radial("ellipse at top") - elliptical gradient at top',
                    'mask_radial("--radial-mask") - CSS custom property'
                ]
            },
            'options': {
                'shape_utilities': 'Use mask_circle, mask_ellipse for shape control',
                'size_utilities': 'Use mask_radial_closest_corner, etc. for size control',
                'position_utilities': 'Use mask_radial_at_center, etc. for position control',
                'from_to_positioning': 'Use mask_radial_from and mask_radial_to for gradient stops'
            }
        }

mask_radial = MaskRadialFactory("Radial gradient mask utilities for creating circular gradient-based masks") # Create the radial gradient mask factory instance

In [ ]:
#| export
# Radial gradient shape controls
mask_circle = SingleValueFactory("mask-circle", "Set radial gradient mask shape to circle")
mask_ellipse = SingleValueFactory("mask-ellipse", "Set radial gradient mask shape to ellipse")

# Radial gradient size controls
mask_radial_closest_corner = SingleValueFactory("mask-radial-closest-corner", "Size radial gradient mask to closest corner")
mask_radial_closest_side = SingleValueFactory("mask-radial-closest-side", "Size radial gradient mask to closest side")
mask_radial_farthest_corner = SingleValueFactory("mask-radial-farthest-corner", "Size radial gradient mask to farthest corner")
mask_radial_farthest_side = SingleValueFactory("mask-radial-farthest-side", "Size radial gradient mask to farthest side")

# Radial gradient position controls
mask_radial_at_top_left = SingleValueFactory("mask-radial-at-top-left", "Position radial gradient mask at top left")
mask_radial_at_top = SingleValueFactory("mask-radial-at-top", "Position radial gradient mask at top")
mask_radial_at_top_right = SingleValueFactory("mask-radial-at-top-right", "Position radial gradient mask at top right")
mask_radial_at_left = SingleValueFactory("mask-radial-at-left", "Position radial gradient mask at left")
mask_radial_at_center = SingleValueFactory("mask-radial-at-center", "Position radial gradient mask at center")
mask_radial_at_right = SingleValueFactory("mask-radial-at-right", "Position radial gradient mask at right")
mask_radial_at_bottom_left = SingleValueFactory("mask-radial-at-bottom-left", "Position radial gradient mask at bottom left")
mask_radial_at_bottom = SingleValueFactory("mask-radial-at-bottom", "Position radial gradient mask at bottom")
mask_radial_at_bottom_right = SingleValueFactory("mask-radial-at-bottom-right", "Position radial gradient mask at bottom right")

# Radial gradient from/to positioning
mask_radial_from = MaskDirectionalFactory('radial', 'from', "Radial gradient mask from position utilities")
mask_radial_to = MaskDirectionalFactory('radial', 'to', "Radial gradient mask to position utilities")

#### Conic Gradient Masks

Create conic gradient masks with angle support:

In [ ]:
#| export
class MaskConicUtility(BaseUtility):
    """Utility class for conic gradient masks with angle support."""
    
    def __init__(
        self,
        angle: Optional[Union[int, str]] = None,  # Starting angle in degrees or custom value
        negative: bool = False  # Whether to negate the angle
    ):
        """Initialize conic gradient mask utility."""
        super().__init__("mask-conic")
        self.negative = negative
        if angle is not None:
            self._value = self._format_value(angle)
    
    def _format_value(
        self,
        value: TailwindValue  # The value to format
    ) -> str:  # The formatted value string
        """Format conic gradient mask value."""
        # Handle numeric angles
        if isinstance(value, int) or (isinstance(value, str) and value.isdigit()):
            return str(value)
        
        value_str = str(value)
        
        # Handle custom properties
        if is_custom_property(value_str):
            return f"({value_str})"
        
        # Handle arbitrary values
        return f"[{value_str}]"
    
    def _build_class(
        self,
        value: Optional[TailwindValue] = None  # Optional value to use
    ) -> str:  # The complete CSS class string
        """Build the conic gradient mask class string."""
        if value is not None:
            self._value = self._format_value(value)
        
        # Conic gradient mask requires an angle
        if not hasattr(self, '_value') or self._value is None:
            raise ValueError("Conic gradient mask requires an angle value")
        
        # Apply negative prefix for angles
        neg_prefix = "-" if self.negative and self._value.isdigit() else ""
        
        # Build the class
        base_class = f"{neg_prefix}{self.prefix}-{self._value}"
        
        # Apply modifiers
        if self._modifiers:
            return ":".join(self._modifiers + [base_class])
        
        return base_class

In [ ]:
#| export
class MaskConicFactory(BaseFactory):
    """Factory for conic gradient mask utilities with angle support."""
    
    def __init__(
        self,
        doc: Optional[str] = None  # Documentation
    ):
        """Initialize conic gradient mask factory."""
        doc = doc or "Conic gradient mask utilities for creating conical gradient-based masks"
        super().__init__(doc)
        
        # Common starting angles
        self._0 = MaskConicUtility(0)
        self._45 = MaskConicUtility(45)
        self._90 = MaskConicUtility(90)
        self._180 = MaskConicUtility(180)
    
    def __call__(
        self,
        angle: Union[int, str],  # Starting angle in degrees or custom value
        negative: bool = False  # Whether to negate angle
    ) -> MaskConicUtility:  # A conic gradient mask utility instance
        """Create conic gradient mask utility with angle."""
        return MaskConicUtility(angle, negative)
    
    def __getattr__(
        self,
        name: str  # Attribute name (angle)
    ) -> MaskConicUtility:  # A conic gradient mask utility instance
        """Handle angle access (e.g., mask_conic._45, mask_conic._90)."""
        # Handle underscore prefix for numeric angles
        if name.startswith("_") and name[1:].isdigit():
            angle = int(name[1:])
            return MaskConicUtility(angle)
        
        # Handle negative angles (e.g., neg_45)
        if name.startswith("neg_") and name[4:].isdigit():
            angle = int(name[4:])
            return MaskConicUtility(angle, negative=True)
        
        raise AttributeError(f"'{self.__class__.__name__}' object has no attribute '{name}'")
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Factory information
        """Get information about the conic gradient mask factory."""
        return {
            'description': self._doc,
            'valid_inputs': {
                'angles': 'Any numeric starting angle (0-360 degrees)',
                'usage_examples': [
                    'mask_conic(0) or mask_conic._0 - start at 0 degrees',
                    'mask_conic(90, negative=True) or mask_conic.neg_90 - negative 90 degree start',
                    'mask_conic._45 - start at 45 degrees',
                    'mask_conic._180 - start at 180 degrees',
                    'mask_conic("--conic-angle") - CSS custom property'
                ]
            },
            'options': {
                'negative_angles': 'Use negative=True or neg_<angle> syntax',
                'from_to_positioning': 'Use mask_conic_from and mask_conic_to for gradient stops'
            }
        }

# Create the conic gradient mask factory instance
mask_conic = MaskConicFactory("Conic gradient mask utilities for creating conical gradient-based masks")

# Conic gradient from/to positioning
mask_conic_from = MaskDirectionalFactory('conic', 'from', "Conic gradient mask from position utilities")
mask_conic_to = MaskDirectionalFactory('conic', 'to', "Conic gradient mask to position utilities")

### Mask Mode

Utilities for controlling an element's mask mode:

In [ ]:
#| export
mask_mode = SimpleFactory(
    {
        "alpha": "mask-alpha",
        "luminance": "mask-luminance",
        "match": "mask-match"
    },
    "Mask mode utilities for controlling how the mask is interpreted"
) # Mask mode factory

### Mask Origin

Utilities for controlling how an element's mask image is positioned relative to borders, padding, and content:

In [ ]:
#| export
mask_origin = SimpleFactory(
    {
        "border": "mask-origin-border",
        "padding": "mask-origin-padding",
        "content": "mask-origin-content",
        "fill": "mask-origin-fill",
        "stroke": "mask-origin-stroke",
        "view": "mask-origin-view"
    },
    "Mask origin utilities for controlling how the mask is positioned relative to borders, padding, and content"
) # Mask origin factory

### Mask Position

Utilities for controlling the position of an element's mask image:

In [ ]:
#| export
mask_position = SimpleFactory(
    {
        "top_left": "mask-top-left",
        "top": "mask-top",
        "top_right": "mask-top-right",
        "left": "mask-left",
        "center": "mask-center",
        "right": "mask-right",
        "bottom_left": "mask-bottom-left",
        "bottom": "mask-bottom",
        "bottom_right": "mask-bottom-right"
    },
    "Mask position utilities for controlling the position of an element's mask image"
) # Mask position factory

### Mask Repeat

Utilities for controlling the repetition of an element's mask image:

In [ ]:
#| export
mask_repeat = SimpleFactory(
    {
        "repeat": "mask-repeat",
        "no_repeat": "mask-no-repeat",
        "repeat_x": "mask-repeat-x",
        "repeat_y": "mask-repeat-y",
        "repeat_space": "mask-repeat-space",
        "repeat_round": "mask-repeat-round"
    },
    "Mask repeat utilities for controlling the repetition of an element's mask image"
) # Mask repeat factory

### Mask Size

Utilities for controlling the size of an element's mask image:

In [ ]:
#| export
mask_size = SimpleFactory(
    {
        "auto": "mask-auto",
        "cover": "mask-cover",
        "contain": "mask-contain"
    },
    "Mask size utilities for controlling the size of an element's mask image"
) # Mask size factory

### Mask Type

Utilities for controlling how an SVG mask is interpreted:

In [ ]:
#| export
mask_type = SimpleFactory(
    {
        "alpha": "mask-type-alpha",
        "luminance": "mask-type-luminance"
    },
    "Mask type utilities for controlling how an SVG mask is interpreted"
) # Mask type factory

### Test Mask Utilities

Test all mask image utilities:

In [ ]:
#| export
def test_effects_mask_basic_examples():
    """Test basic mask image utilities."""
    # Test mask-none
    assert str(mask.none) == "mask-none"
    
    # Test arbitrary mask values
    assert str(mask("url(#my-mask)")) == "mask-[url(#my-mask)]"
    assert str(mask("linear-gradient(black, transparent)")) == "mask-[linear-gradient(black, transparent)]"
    assert str(mask("radial-gradient(circle, black, transparent)")) == "mask-[radial-gradient(circle, black, transparent)]"
    
    # Test CSS custom properties
    assert str(mask("--custom-mask")) == "mask-(--custom-mask)"
    assert str(mask("--theme-mask")) == "mask-(--theme-mask)"

# Run the tests
test_effects_mask_basic_examples()

In [ ]:
#| export
def test_effects_mask_linear_examples():
    """Test linear gradient mask utilities."""
    # Test common angles
    assert str(mask_linear._0) == "mask-linear-0"
    assert str(mask_linear._45) == "mask-linear-45"
    assert str(mask_linear._90) == "mask-linear-90"
    assert str(mask_linear._180) == "mask-linear-180"
    
    # Test with angle parameter
    assert str(mask_linear(30)) == "mask-linear-30"
    assert str(mask_linear(60)) == "mask-linear-60"
    assert str(mask_linear(270)) == "mask-linear-270"
    
    # Test negative angles
    assert str(mask_linear(45, negative=True)) == "-mask-linear-45"
    assert str(mask_linear.neg_90) == "-mask-linear-90"
    assert str(mask_linear.neg_180) == "-mask-linear-180"
    
    # Test CSS custom properties
    assert str(mask_linear("--gradient-angle")) == "mask-linear-(--gradient-angle)"

# Run the tests
test_effects_mask_linear_examples()

In [ ]:
#| export
def test_effects_mask_directional_examples():
    """Test directional mask gradient utilities."""
    # Test top direction
    assert str(mask_t_from(50)) == "mask-t-from-50"
    assert str(mask_t_from("50%")) == "mask-t-from-50%"
    assert str(mask_t_from.black) == "mask-t-from-black"
    assert str(mask_t_to(100)) == "mask-t-to-100"
    assert str(mask_t_to.transparent) == "mask-t-to-transparent"
    
    # Test right direction
    assert str(mask_r_from._20) == "mask-r-from-20"
    assert str(mask_r_from._50p) == "mask-r-from-50%"
    assert str(mask_r_to.white) == "mask-r-to-white"
    
    # Test bottom direction
    assert str(mask_b_from(0)) == "mask-b-from-0"
    assert str(mask_b_to("75%")) == "mask-b-to-75%"
    
    # Test left direction
    assert str(mask_l_from(10)) == "mask-l-from-10"
    assert str(mask_l_to(90)) == "mask-l-to-90"
    
    # Test horizontal (x) direction
    assert str(mask_x_from(25)) == "mask-x-from-25"
    assert str(mask_x_to("100%")) == "mask-x-to-100%"
    
    # Test vertical (y) direction
    assert str(mask_y_from(0)) == "mask-y-from-0"
    assert str(mask_y_to(50)) == "mask-y-to-50"
    
    # Test with custom properties
    assert str(mask_t_from("--mask-start")) == "mask-t-from-(--mask-start)"
    assert str(mask_r_to("--mask-end")) == "mask-r-to-(--mask-end)"
    
    # Test with arbitrary values
    assert str(mask_b_from("10px")) == "mask-b-from-[10px]"
    assert str(mask_l_to("5rem")) == "mask-l-to-[5rem]"

# Run the tests
test_effects_mask_directional_examples()

In [ ]:
#| export
def test_effects_mask_radial_examples():
    """Test radial gradient mask utilities."""
    # Test basic radial gradient
    assert str(mask_radial()) == "mask-radial"
    
    # Test arbitrary radial values
    assert str(mask_radial("circle at center")) == "mask-radial-[circle at center]"
    assert str(mask_radial("ellipse at top")) == "mask-radial-[ellipse at top]"
    assert str(mask_radial("circle 50px at 25% 75%")) == "mask-radial-[circle 50px at 25% 75%]"
    
    # Test shape controls
    assert str(mask_circle) == "mask-circle"
    assert str(mask_ellipse) == "mask-ellipse"
    
    # Test size controls
    assert str(mask_radial_closest_corner) == "mask-radial-closest-corner"
    assert str(mask_radial_closest_side) == "mask-radial-closest-side"
    assert str(mask_radial_farthest_corner) == "mask-radial-farthest-corner"
    assert str(mask_radial_farthest_side) == "mask-radial-farthest-side"
    
    # Test position controls
    assert str(mask_radial_at_top_left) == "mask-radial-at-top-left"
    assert str(mask_radial_at_top) == "mask-radial-at-top"
    assert str(mask_radial_at_top_right) == "mask-radial-at-top-right"
    assert str(mask_radial_at_center) == "mask-radial-at-center"
    assert str(mask_radial_at_bottom) == "mask-radial-at-bottom"
    
    # Test radial from/to
    assert str(mask_radial_from(0)) == "mask-radial-from-0"
    assert str(mask_radial_from("25%")) == "mask-radial-from-25%"
    assert str(mask_radial_from.black) == "mask-radial-from-black"
    assert str(mask_radial_to(100)) == "mask-radial-to-100"
    assert str(mask_radial_to.transparent) == "mask-radial-to-transparent"
    
    # Test CSS custom properties
    assert str(mask_radial("--radial-mask")) == "mask-radial-(--radial-mask)"

# Run the tests
test_effects_mask_radial_examples()

In [ ]:
#| export
def test_effects_mask_conic_examples():
    """Test conic gradient mask utilities."""
    # Test common angles
    assert str(mask_conic._0) == "mask-conic-0"
    assert str(mask_conic._45) == "mask-conic-45"
    assert str(mask_conic._90) == "mask-conic-90"
    assert str(mask_conic._180) == "mask-conic-180"
    
    # Test with angle parameter
    assert str(mask_conic(30)) == "mask-conic-30"
    assert str(mask_conic(120)) == "mask-conic-120"
    assert str(mask_conic(270)) == "mask-conic-270"
    
    # Test negative angles
    assert str(mask_conic(45, negative=True)) == "-mask-conic-45"
    assert str(mask_conic.neg_90) == "-mask-conic-90"
    assert str(mask_conic.neg_180) == "-mask-conic-180"
    
    # Test conic from/to
    assert str(mask_conic_from(0)) == "mask-conic-from-0"
    assert str(mask_conic_from("25%")) == "mask-conic-from-25%"
    assert str(mask_conic_from.black) == "mask-conic-from-black"
    assert str(mask_conic_to(180)) == "mask-conic-to-180"
    assert str(mask_conic_to.transparent) == "mask-conic-to-transparent"
    
    # Test CSS custom properties
    assert str(mask_conic("--conic-angle")) == "mask-conic-(--conic-angle)"

# Run the tests
test_effects_mask_conic_examples()

In [ ]:
#| export
def test_effects_mask_properties_examples():
    """Test mask property utilities (mode, origin, position, etc.)."""
    # Test mask mode
    assert str(mask_mode.alpha) == "mask-alpha"
    assert str(mask_mode.luminance) == "mask-luminance"
    assert str(mask_mode.match) == "mask-match"
    
    # Test mask origin
    assert str(mask_origin.border) == "mask-origin-border"
    assert str(mask_origin.padding) == "mask-origin-padding"
    assert str(mask_origin.content) == "mask-origin-content"
    assert str(mask_origin.fill) == "mask-origin-fill"
    assert str(mask_origin.stroke) == "mask-origin-stroke"
    assert str(mask_origin.view) == "mask-origin-view"
    
    # Test mask position
    assert str(mask_position.top_left) == "mask-top-left"
    assert str(mask_position.top) == "mask-top"
    assert str(mask_position.center) == "mask-center"
    assert str(mask_position.bottom_right) == "mask-bottom-right"
    
    # Test mask repeat
    assert str(mask_repeat.repeat) == "mask-repeat"
    assert str(mask_repeat.no_repeat) == "mask-no-repeat"
    assert str(mask_repeat.repeat_x) == "mask-repeat-x"
    assert str(mask_repeat.repeat_y) == "mask-repeat-y"
    assert str(mask_repeat.repeat_space) == "mask-repeat-space"
    assert str(mask_repeat.repeat_round) == "mask-repeat-round"
    
    # Test mask size
    assert str(mask_size.auto) == "mask-auto"
    assert str(mask_size.cover) == "mask-cover"
    assert str(mask_size.contain) == "mask-contain"
    
    # Test mask type
    assert str(mask_type.alpha) == "mask-type-alpha"
    assert str(mask_type.luminance) == "mask-type-luminance"

# Run the tests
test_effects_mask_properties_examples()

## Practical Examples

Let's see how to use shadow utilities in real FastHTML components:

In [ ]:
#| export
def test_effects_shadow_practical_examples():
    """Test shadow utilities in practical FastHTML component examples."""
    from fasthtml.common import Div, H2, P, Button, Card
    
    # Card with basic shadow
    card = Div(
        H2("Card Title"),
        P("Card content goes here."),
        cls=combine_classes(shadow.md, "p-6 bg-white rounded-lg")
    )
    assert "shadow-md" in card.attrs['class']
    
    # Button with hover shadow effect
    button = Button(
        "Click me",
        cls=combine_classes(
            shadow.sm,
            "hover:shadow-lg",
            "transition-shadow",
            "px-4 py-2 bg-blue-500 text-white rounded"
        )
    )
    assert "shadow-sm" in button.attrs['class']
    
    # Card with colored shadow
    colored_card = Div(
        "Content with colored shadow",
        cls=combine_classes(
            shadow.xl,
            shadow_color.blue._500,
            "p-8 bg-white rounded-xl"
        )
    )
    assert "shadow-xl" in colored_card.attrs['class']
    assert "shadow-blue-500" in colored_card.attrs['class']
    
    # Floating action button with large shadow
    fab = Button(
        "+",
        cls=combine_classes(
            shadow._2xl,
            shadow_color.black,
            "w-14 h-14 rounded-full bg-purple-600 text-white"
        )
    )
    assert "shadow-2xl" in fab.attrs['class']
    assert "shadow-black" in fab.attrs['class']
    
    # Card with no shadow (flat design)
    flat_card = Div(
        "Flat design card",
        cls=combine_classes(
            shadow.none,
            "p-4 bg-gray-100 border border-gray-300"
        )
    )
    assert "shadow-none" in flat_card.attrs['class']
    
    # Card with custom shadow
    custom_shadow_card = Div(
        "Custom shadow",
        cls=combine_classes(
            shadow("0 10px 30px -10px rgba(0, 0, 0, 0.3)"),
            "p-6 bg-white rounded"
        )
    )
    assert "shadow-[0 10px 30px -10px rgba(0, 0, 0, 0.3)]" in custom_shadow_card.attrs['class']
    
    # Input with inset shadow for depth
    input_field = Div(
        cls=combine_classes(
            inset_shadow.sm,
            inset_shadow_color.gray._400,
            "p-3 bg-gray-50 rounded border border-gray-300"
        )
    )
    assert "inset-shadow-sm" in input_field.attrs['class']
    assert "inset-shadow-gray-400" in input_field.attrs['class']
    
    # Button with focus ring
    focus_button = Button(
        "Focus me",
        cls=combine_classes(
            "focus:ring-2",
            "focus:ring-blue-500",
            "px-4 py-2 bg-gray-200 rounded"
        )
    )
    assert "focus:ring-2" in focus_button.attrs['class']
    
    # Card with custom ring
    ring_card = Div(
        "Ring example",
        cls=combine_classes(
            ring._2,
            ring_color.purple._500,
            "p-6 bg-white rounded"
        )
    )
    assert "ring-2" in ring_card.attrs['class']
    assert "ring-purple-500" in ring_card.attrs['class']

# Run the tests
test_effects_shadow_practical_examples()

In [ ]:
#| export
def test_effects_shadow_composition_examples():
    """Test composing shadow size and color utilities."""
    from fasthtml.common import Div
    
    # Small shadow with custom color
    example1 = Div(
        "Small blue shadow",
        cls=combine_classes(
            shadow.sm,
            shadow_color.blue._400,
            "p-4 bg-white"
        )
    )
    assert "shadow-sm" in example1.attrs['class']
    assert "shadow-blue-400" in example1.attrs['class']
    
    # Large shadow with red color
    example2 = Div(
        "Large red shadow",
        cls=combine_classes(
            shadow.xl,
            shadow_color.red._600,
            "p-6 bg-white"
        )
    )
    assert "shadow-xl" in example2.attrs['class']
    assert "shadow-red-600" in example2.attrs['class']
    
    # Extra small shadow with transparent color
    example3 = Div(
        "Subtle transparent shadow",
        cls=combine_classes(
            shadow._2xs,
            shadow_color.transparent,
            "p-3 bg-gray-50"
        )
    )
    assert "shadow-2xs" in example3.attrs['class']
    assert "shadow-transparent" in example3.attrs['class']
    
    # Medium shadow with custom hex color
    example4 = Div(
        "Custom color shadow",
        cls=combine_classes(
            shadow.md,
            shadow_color("#6B46C1"),  # Purple hex
            "p-5 bg-white"
        )
    )
    assert "shadow-md" in example4.attrs['class']
    assert "shadow-[#6B46C1]" in example4.attrs['class']

# Run the tests
test_effects_shadow_composition_examples()

In [ ]:
#| export
def test_effects_comprehensive_examples():
    """Test comprehensive usage of all effect utilities."""
    from fasthtml.common import Div, Input, Button
    
    # Form input with inset shadow
    form_input = Input(
        type="text",
        placeholder="Enter text",
        cls=combine_classes(
            inset_shadow.xs,
            inset_shadow_color.gray._300,
            "px-4 py-2 bg-white border border-gray-200 rounded-md",
            "focus:inset-shadow-sm",
            "focus:inset-shadow-blue-400"
        )
    )
    assert "inset-shadow-xs" in form_input.attrs['class']
    assert "inset-shadow-gray-300" in form_input.attrs['class']
    
    # Button with ring on focus
    ring_button = Button(
        "Click for Ring",
        cls=combine_classes(
            "px-6 py-3 bg-blue-500 text-white rounded-lg",
            "focus:ring-4",
            "focus:ring-blue-300",
            "focus:ring-offset-2"
        )
    )
    assert "focus:ring-4" in ring_button.attrs['class']
    
    # Card with inset ring
    inset_ring_card = Div(
        "Inset Ring Card",
        cls=combine_classes(
            inset_ring._2,
            inset_ring_color.indigo._400,
            "p-8 bg-white rounded-xl"
        )
    )
    assert "inset-ring-2" in inset_ring_card.attrs['class']
    assert "inset-ring-indigo-400" in inset_ring_card.attrs['class']
    
    # Complex shadow layering
    layered_card = Div(
        "Layered Effects",
        cls=combine_classes(
            shadow.lg,
            shadow_color.purple._500,
            inset_shadow.sm,
            inset_shadow_color.purple._200,
            "p-6 bg-gradient-to-br from-purple-100 to-purple-50 rounded-lg"
        )
    )
    assert "shadow-lg" in layered_card.attrs['class']
    assert "shadow-purple-500" in layered_card.attrs['class']
    assert "inset-shadow-sm" in layered_card.attrs['class']
    assert "inset-shadow-purple-200" in layered_card.attrs['class']
    
    # Remove all effects
    no_effects = Div(
        "No Effects",
        cls=combine_classes(
            shadow.none,
            inset_shadow.none,
            ring._0,
            inset_ring._0,
            "p-4 bg-gray-100"
        )
    )
    assert "shadow-none" in no_effects.attrs['class']
    assert "inset-shadow-none" in no_effects.attrs['class']
    assert "ring-0" in no_effects.attrs['class']
    assert "inset-ring-0" in no_effects.attrs['class']

# Run the tests
test_effects_comprehensive_examples()

## Documentation Tests

Test that all factories provide proper documentation:

## Practical Mask Examples

Let's see how to use mask utilities in real FastHTML components:

In [ ]:
#| export
def test_effects_mask_practical_examples():
    """Test mask utilities in practical FastHTML component examples."""
    from fasthtml.common import Div, Img, Section, H1, P
    
    # Fade out effect with linear gradient mask
    fade_out_div = Div(
        P("This text fades out at the bottom"),
        cls=combine_classes(
            mask_linear._180,
            mask_t_from.black,
            mask_t_to.transparent,
            "p-8 bg-blue-500 text-white"
        )
    )
    assert "mask-linear-180" in fade_out_div.attrs['class']
    assert "mask-t-from-black" in fade_out_div.attrs['class']
    assert "mask-t-to-transparent" in fade_out_div.attrs['class']
    
    # Circular reveal with radial gradient mask
    circular_reveal = Div(
        Img(src="/image.jpg", alt="Masked image"),
        cls=combine_classes(
            mask_radial(),
            mask_circle,
            mask_radial_at_center,
            mask_radial_from.black,
            mask_radial_to.transparent,
            "relative overflow-hidden"
        )
    )
    assert "mask-radial" in circular_reveal.attrs['class']
    assert "mask-circle" in circular_reveal.attrs['class']
    assert "mask-radial-at-center" in circular_reveal.attrs['class']
    
    # Horizontal fade edges effect
    fade_edges = Div(
        "Content with faded edges",
        cls=combine_classes(
            mask_x_from._10,
            mask_x_to._90,
            "px-8 py-4 bg-gradient-to-r from-purple-500 to-pink-500 text-white"
        )
    )
    assert "mask-x-from-10" in fade_edges.attrs['class']
    assert "mask-x-to-90" in fade_edges.attrs['class']
    
    # Corner spotlight effect with radial mask
    corner_spotlight = Section(
        H1("Spotlight Effect"),
        cls=combine_classes(
            mask_radial(),
            mask_ellipse,
            mask_radial_at_top_left,
            mask_radial_farthest_corner,
            mask_radial_from.black,
            mask_radial_to.transparent,
            "p-16 bg-gray-900 text-white"
        )
    )
    assert "mask-ellipse" in corner_spotlight.attrs['class']
    assert "mask-radial-at-top-left" in corner_spotlight.attrs['class']
    assert "mask-radial-farthest-corner" in corner_spotlight.attrs['class']
    
    # SVG mask reference
    svg_masked = Div(
        "Content with SVG mask",
        cls=combine_classes(
            mask("url(#star-mask)"),
            mask_size.cover,
            mask_position.center,
            mask_repeat.no_repeat,
            "p-8 bg-gradient-to-br from-yellow-400 to-orange-500"
        )
    )
    assert "mask-[url(#star-mask)]" in svg_masked.attrs['class']
    assert "mask-cover" in svg_masked.attrs['class']
    assert "mask-center" in svg_masked.attrs['class']
    assert "mask-no-repeat" in svg_masked.attrs['class']
    
    # Conic gradient pie chart mask
    pie_mask = Div(
        "Pie chart effect",
        cls=combine_classes(
            mask_conic._0,
            mask_conic_from.black,
            mask_conic_to.transparent,
            "w-32 h-32 bg-blue-600 rounded-full"
        )
    )
    assert "mask-conic-0" in pie_mask.attrs['class']
    assert "mask-conic-from-black" in pie_mask.attrs['class']
    
    # Complex gradient mask composition
    complex_mask = Div(
        "Complex masked content",
        cls=combine_classes(
            mask_linear._45,
            mask_t_from._20,
            mask_b_from._20,
            mask_l_from("10%"),
            mask_r_to("90%"),
            mask_mode.alpha,
            mask_origin.content,
            "p-12 bg-gradient-to-r from-indigo-500 via-purple-500 to-pink-500 text-white"
        )
    )
    assert "mask-linear-45" in complex_mask.attrs['class']
    assert "mask-t-from-20" in complex_mask.attrs['class']
    assert "mask-alpha" in complex_mask.attrs['class']
    assert "mask-origin-content" in complex_mask.attrs['class']

# Run the tests
test_effects_mask_practical_examples()

In [ ]:
#| export
def test_effects_factory_documentation():
    """Test that effect factories have accessible documentation."""
    # Test shadow factories
    assert shadow.describe() == "Box shadow utilities for adding depth to elements"
    assert shadow_color.describe() == "Shadow color utilities for customizing the color of box shadows"
    
    # Test inset shadow factories
    assert inset_shadow.describe() == "Inset box shadow utilities for adding inner depth to elements"
    assert inset_shadow_color.describe() == "Inset shadow color utilities for customizing the color of inner box shadows"
    
    # Test ring factories
    assert ring.describe() == "Ring (outline) utilities for adding focus rings and outlines to elements"
    assert ring_color.describe() == "Ring color utilities for customizing the color of focus rings and outlines"
    
    # Test inset ring factories
    assert inset_ring.describe() == "Inset ring (inner outline) utilities for adding inner focus rings and outlines to elements"
    assert inset_ring_color.describe() == "Inset ring color utilities for customizing the color of inner focus rings and outlines"
    
    # Test get_info methods
    shadow_info = shadow.get_info()
    assert 'sizes' in shadow_info['valid_inputs']
    assert '2xs' in shadow_info['valid_inputs']['sizes']
    assert 'none' in shadow_info['valid_inputs']['sizes']
    
    inset_shadow_info = inset_shadow.get_info()
    assert 'sizes' in inset_shadow_info['valid_inputs']
    assert 'usage_examples' in inset_shadow_info['valid_inputs']
    
    ring_info = ring.get_info()
    assert 'widths' in ring_info['valid_inputs']
    assert 'usage_examples' in ring_info['valid_inputs']
    
    inset_ring_info = inset_ring.get_info()
    assert 'widths' in inset_ring_info['valid_inputs']
    assert 'usage_examples' in inset_ring_info['valid_inputs']
    
    # Test color factory info
    shadow_color_info = shadow_color.get_info()
    assert 'color_families' in shadow_color_info['valid_inputs']
    assert len(shadow_color_info['valid_inputs']['color_families']) == 22
    
    inset_shadow_color_info = inset_shadow_color.get_info()
    assert inset_shadow_color_info['options']['prefix'] == 'inset-shadow'
    
    ring_color_info = ring_color.get_info()
    assert ring_color_info['options']['prefix'] == 'ring'
    
    inset_ring_color_info = inset_ring_color.get_info()
    assert inset_ring_color_info['options']['prefix'] == 'inset-ring'

# Run the tests
test_effects_factory_documentation()

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()